<a href="https://colab.research.google.com/github/somynt/Assignment_2_LLM_Sem4/blob/main/Assignment_2_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade transformers accelerate
!pip install optuna
!pip install scikit-learn
!pip install seaborn
!pip install lime scikit-learn imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=e11f7e6dd0989d90010a0b0caac2645e797aefce5a31b4f03020110a423b9055
  Stored in directory: /root/.cache/pip/wheels/85/fa/a3/9c2d44c9f3cd77cf4e533b58900b2bf4487f2a17e8ec212a3d
Successfully built lime


In [ ]:
!pip show transformers
!pip show accelerate

Name: transformers
Version: 4.53.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers
Name: accelerate
Version: 1.8.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.11/dist-packages
Requires: huggingface_hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: peft


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
import os
os.makedirs('/root/.kaggle', exist_ok=True)
!cp kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!ls -l /root/.kaggle/kaggle.json

-rw------- 1 root root 66 Jul  9 18:21 /root/.kaggle/kaggle.json


In [ ]:
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge

jigsaw-toxic-comment-classification-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile

with zipfile.ZipFile("jigsaw-toxic-comment-classification-challenge.zip", "r") as zip_ref:
    zip_ref.extractall()

In [ ]:
import zipfile

zip_files = ['train.csv.zip', 'test.csv.zip', 'test_labels.csv.zip', 'sample_submission.csv.zip']

for zf in zip_files:
    with zipfile.ZipFile(zf, 'r') as zip_ref:
        zip_ref.extractall()

In [ ]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
test_labels = pd.read_csv("test_labels.csv")
sample_submission = pd.read_csv("sample_submission.csv")

print(train_df.shape)
print(test_df.shape)

(159571, 8)
(153164, 2)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

warnings.filterwarnings('ignore')

# --- Your Data Loading from Kaggle ---
# Ensure these CSV files are accessible in your Google Colab environment
# (e.g., mounted Google Drive or uploaded directly)
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
test_labels = pd.read_csv("test_labels.csv")
sample_submission = pd.read_csv("sample_submission.csv")

print(f"Train DataFrame shape: {train_df.shape}")
print(f"Test DataFrame shape: {test_df.shape}")
print(f"Test Labels DataFrame shape: {test_labels.shape}")
print(f"Sample Submission DataFrame shape: {sample_submission.shape}")

# --- End of Your Data Loading ---


# Set global style for better plots
# Changed to 'seaborn-v0_8-whitegrid' for a white background with grid lines
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis") # A colorblind-friendly and visually appealing palette

print("\n" + "=" * 60)
print("TOXIC COMMENT CLASSIFICATION - DATASET EXPLORATION")
print("=" * 60)

# 1. BASIC DATASET INFORMATION
print("\n" + "=" * 40)
print("1. BASIC DATASET INFORMATION")
print("=" * 40)

print(f"Dataset shape: {train_df.shape}")
print(f"Columns: {train_df.columns.tolist()}")
print(f"Memory usage: {train_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Check data types
print(f"\nData types:")
print(train_df.dtypes)

# Missing values analysis
print(f"\nMissing values:")
missing_vals = train_df.isnull().sum()
if missing_vals.any():
    print(missing_vals[missing_vals > 0])
else:
    print("No missing values found!")

# 2. SAMPLE DATA EXAMINATION
print("\n" + "=" * 40)
print("2. SAMPLE DATA EXAMINATION")
print("=" * 40)

print("\nFirst 3 rows:")
for idx, row in train_df.head(3).iterrows():
    print(f"\nRow {idx}:")
    print(f"Comment: {row['comment_text'][:100]}...")
    labels = [col for col in train_df.columns[2:] if row[col] == 1]
    print(f"Labels: {labels if labels else 'None (clean)'}")

# 3. TARGET LABELS ANALYSIS
print("\n" + "=" * 40)
print("3. TARGET LABELS ANALYSIS")
print("=" * 40)

label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

if not all(col in train_df.columns for col in label_cols):
    print(f"Warning: Not all expected label columns found in train_df. Found: {train_df.columns.tolist()}")
    label_cols = train_df.columns[2:].tolist()
    print(f"Using dynamically detected label columns: {label_cols}")

print(f"Label columns: {label_cols}")

label_stats = {}
for col in label_cols:
    positive_count = train_df[col].sum()
    positive_pct = (positive_count / len(train_df)) * 100
    label_stats[col] = {'count': positive_count, 'percentage': positive_pct}
    print(f"{col}: {positive_count:,} ({positive_pct:.2f}%)")

train_df['is_toxic'] = (train_df[label_cols].sum(axis=1) > 0).astype(int)
toxic_rate = train_df['is_toxic'].mean() * 100
print(f"\nOverall toxic comments: {train_df['is_toxic'].sum():,} ({toxic_rate:.2f}%)")
print(f"Clean comments: {(~train_df['is_toxic'].astype(bool)).sum():,} ({100-toxic_rate:.2f}%)")

# 4. COMMENT LENGTH ANALYSIS
print("\n" + "=" * 40)
print("4. COMMENT LENGTH ANALYSIS")
print("=" * 40)

train_df['char_length'] = train_df['comment_text'].apply(len)
train_df['word_count'] = train_df['comment_text'].apply(lambda x: len(str(x).split()))
train_df['sentence_count'] = train_df['comment_text'].apply(lambda x: len(re.split(r'[.!?]+', str(x))))

length_metrics = ['char_length', 'word_count', 'sentence_count']
print("Length statistics:")
for metric in length_metrics:
    stats = train_df[metric].describe()
    print(f"\n{metric.replace('_', ' ').title()}:")
    print(f"  Mean: {stats['mean']:.1f} | Median: {stats['50%']:.1f}")
    print(f"  Min: {stats['min']:.0f} | Max: {stats['max']:.0f}")
    print(f"  Std: {stats['std']:.1f}")

print(f"\nLength comparison (Toxic vs Clean):")
for metric in ['char_length', 'word_count']:
    toxic_mean = train_df[train_df['is_toxic'] == 1][metric].mean()
    clean_mean = train_df[train_df['is_toxic'] == 0][metric].mean()
    print(f"{metric}: Toxic={toxic_mean:.1f}, Clean={clean_mean:.1f}")

# 5. LABEL CORRELATION ANALYSIS
print("\n" + "=" * 40)
print("5. LABEL CORRELATION ANALYSIS")
print("=" * 40)

label_corr = train_df[label_cols].corr()
print("Label correlations (top pairs):")
corr_pairs = []
for i in range(len(label_cols)):
    for j in range(i + 1, len(label_cols)):
        corr_val = label_corr.iloc[i, j]
        corr_pairs.append((label_cols[i], label_cols[j], corr_val))

corr_pairs.sort(key=lambda x: abs(x[2]), reverse=True)
for label1, label2, corr in corr_pairs[:5]:
    print(f"{label1} - {label2}: {corr:.3f}")

print(f"\nMulti-label statistics:")
train_df['label_count'] = train_df[label_cols].sum(axis=1)
label_count_dist = train_df['label_count'].value_counts().sort_index()
for count, freq in label_count_dist.items():
    pct = (freq / len(train_df)) * 100
    print(f"{count} labels: {freq:,} ({pct:.2f}%)")

# 6. TEXT CHARACTERISTICS ANALYSIS
print("\n" + "=" * 40)
print("6. TEXT CHARACTERISTICS ANALYSIS")
print("=" * 40)

def analyze_text_patterns(df, toxic_flag):
    subset = df[df['is_toxic'] == toxic_flag]['comment_text']

    metrics = {
        'avg_uppercase_ratio': subset.apply(lambda x: sum(c.isupper() for c in str(x)) / max(len(str(x)), 1)).mean(),
        'avg_exclamation_marks': subset.apply(lambda x: str(x).count('!')).mean(),
        'avg_question_marks': subset.apply(lambda x: str(x).count('?')).mean(),
        'avg_special_chars': subset.apply(lambda x: len(re.findall(r'[^a-zA-Z0-9\s]', str(x)))).mean(),
    }
    return metrics

toxic_patterns = analyze_text_patterns(train_df, 1)
clean_patterns = analyze_text_patterns(train_df, 0)

print("Text pattern comparison:")
for pattern in toxic_patterns:
    print(f"{pattern}: Toxic={toxic_patterns[pattern]:.3f}, Clean={clean_patterns[pattern]:.3f}")

# 7. VISUALIZATION SECTION - GENERATING INDIVIDUAL PLOTS
print("\n" + "=" * 40)
print("7. GENERATING INDIVIDUAL VISUALIZATIONS FOR POSTER")
print("=" * 40)

# Function to save plots
def save_plot(fig, filename):
    """Saves the given figure to a file."""
    fig.savefig(filename, bbox_inches='tight', dpi=300, facecolor=fig.get_facecolor()) # Ensure background is saved
    plt.close(fig) # Close the figure to free memory

# --- Individual Plot Functions ---

# Removed plot_label_distribution_counts and plot_label_distribution_percentages as requested

def plot_comment_length_distribution(train_df, filename="comment_length_distribution.png"):
    fig, ax = plt.subplots(figsize=(12, 7))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')
    sns.histplot(train_df['word_count'], bins=50, kde=True, color='teal', edgecolor='black', ax=ax)
    ax.axvline(train_df['word_count'].mean(), color='red', linestyle='--', label=f'Mean: {train_df["word_count"].mean():.1f}', linewidth=2)
    ax.axvline(train_df['word_count'].median(), color='orange', linestyle='--', label=f'Median: {train_df["word_count"].median():.1f}', linewidth=2)
    ax.set_title('Comment Length Distribution (Word Count)', fontweight='bold', fontsize=16)
    ax.set_xlabel('Word Count', fontsize=14)
    ax.set_ylabel('Frequency', fontsize=14)
    ax.tick_params(axis='both', labelsize=12)
    ax.legend(fontsize=12)
    ax.set_xlim(0, train_df['word_count'].quantile(0.98))
    save_plot(fig, filename)
    print(f"Saved: {filename}")

def plot_toxic_vs_clean_length(train_df, filename="toxic_vs_clean_length.png"):
    fig, ax = plt.subplots(figsize=(10, 7))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')
    sns.boxplot(x='is_toxic', y='word_count', data=train_df, palette=['lightgreen', 'lightcoral'], ax=ax)
    ax.set_xticklabels(['Clean', 'Toxic'], fontsize=12)
    ax.set_title('Comment Length by Toxicity (Word Count)', fontweight='bold', fontsize=16)
    ax.set_xlabel('Comment Type', fontsize=14)
    ax.set_ylabel('Word Count (Log Scale)', fontsize=14)
    ax.tick_params(axis='y', labelsize=12)
    ax.set_yscale('log')
    save_plot(fig, filename)
    print(f"Saved: {filename}")

def plot_label_correlation_heatmap(label_corr, filename="label_correlation_heatmap.png"):
    fig, ax = plt.subplots(figsize=(10, 9))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')
    mask = np.triu(np.ones_like(label_corr, dtype=bool))
    sns.heatmap(label_corr, annot=True, cmap='coolwarm', center=0, fmt=".2f",
                linewidths=.5, linecolor='black', cbar_kws={"shrink": .8}, mask=mask,
                annot_kws={"size": 10}, ax=ax)
    ax.set_title('Label Correlation Heatmap', fontweight='bold', fontsize=16)
    ax.set_xticklabels([col.replace('_', ' ').title() for col in label_corr.columns], rotation=45, ha='right', fontsize=12)
    ax.set_yticklabels([col.replace('_', ' ').title() for col in label_corr.columns], rotation=0, fontsize=12)

    # --- UPDATED: Concise summary textbox moved to the top-right corner ---
    ax.text(0.98, 0.98,
            "Key Label Overlaps:\n"
            "- Insult & Obscene: Strongest (~0.74)\n"
            "- Toxic: High with Obscene & Insult (~0.65-0.68)\n"
            "- Threat & Identity Hate: Generally low overlap",
            transform=ax.transAxes,
            fontsize=10, verticalalignment='top', horizontalalignment='right', # Adjusted va to 'top'
            bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.7, ec='k', lw=0.5))
    # ---------------------------------------------------------------------

    save_plot(fig, filename)
    print(f"Saved: {filename}")

def plot_multi_label_distribution(train_df, label_cols, filename="multi_label_distribution.png"):
    fig, ax = plt.subplots(figsize=(10, 7))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')
    train_df['label_count'] = train_df[label_cols].sum(axis=1)
    label_count_dist = train_df['label_count'].value_counts().sort_index()
    sns.barplot(x=label_count_dist.index, y=label_count_dist.values, color='steelblue', ax=ax)
    ax.set_title('Distribution of Number of Labels per Comment', fontweight='bold', fontsize=16)
    ax.set_xlabel('Number of Labels (0 = Clean)', fontsize=14)
    ax.set_ylabel('Number of Comments', fontsize=14)
    ax.tick_params(axis='both', labelsize=12)
    for i, v in enumerate(label_count_dist.values):
        ax.text(label_count_dist.index[i], v + max(label_count_dist.values) * 0.01,
                f'{v:,}', ha='center', va='bottom', fontsize=10)
    save_plot(fig, filename)
    print(f"Saved: {filename}")

def plot_overall_toxicity_distribution(train_df, filename="overall_toxicity_distribution.png"):
    fig, ax = plt.subplots(figsize=(8, 8))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')
    overall_toxic_dist = train_df['is_toxic'].value_counts()
    ax.pie(overall_toxic_dist, labels=['Clean', 'Toxic'], autopct='%1.1f%%',
           colors=['lightgreen', 'lightcoral'], startangle=90, explode=(0, 0.05),
           textprops={'fontsize': 14})
    ax.set_title('Overall Toxicity Distribution', fontweight='bold', fontsize=16)
    ax.axis('equal')
    save_plot(fig, filename)
    print(f"Saved: {filename}")

def plot_avg_length_per_toxic_label(train_df, label_cols, filename="avg_length_per_toxic_label.png"):
    fig, ax = plt.subplots(figsize=(10, 7))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')
    avg_length_per_toxic_label = {}
    for col in label_cols:
        subset = train_df[(train_df[col] == 1) & (train_df['is_toxic'] == 1)]
        if not subset.empty:
            avg_length_per_toxic_label[col] = subset['word_count'].mean()
        else:
            avg_length_per_toxic_label[col] = 0

    avg_length_df = pd.DataFrame(avg_length_per_toxic_label.items(), columns=['Label', 'Average Word Count'])
    avg_length_df = avg_length_df.sort_values(by='Average Word Count', ascending=False)

    sns.barplot(x='Average Word Count', y='Label', data=avg_length_df, palette='viridis', ax=ax)
    ax.set_title('Average Comment Length per Toxic Label', fontweight='bold', fontsize=16)
    ax.set_xlabel('Average Word Count', fontsize=14)
    ax.set_ylabel('Toxic Label', fontsize=14)
    ax.tick_params(axis='both', labelsize=12)
    save_plot(fig, filename)
    print(f"Saved: {filename}")

def plot_text_characteristics_comparison(toxic_patterns, clean_patterns, filename="text_characteristics_comparison.png"):
    fig, ax = plt.subplots(figsize=(12, 7))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')
    patterns_df = pd.DataFrame({
        'Metric': [m.replace('_', ' ').title() for m in toxic_patterns.keys()],
        'Toxic': list(toxic_patterns.values()),
        'Clean': list(clean_patterns.values())
    })
    patterns_df_melted = patterns_df.melt(id_vars='Metric', var_name='Comment Type', value_name='Average Value')

    sns.barplot(x='Metric', y='Average Value', hue='Comment Type', data=patterns_df_melted, palette=['lightcoral', 'lightgreen'], ax=ax)
    ax.set_title('Average Text Characteristics: Toxic vs Clean', fontweight='bold', fontsize=16)
    ax.set_xlabel('Text Characteristic', fontsize=14)
    ax.set_ylabel('Average Value', fontsize=14)
    ax.tick_params(axis='y', labelsize=12)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=12)
    ax.legend(title='Comment Type', fontsize=12, title_fontsize=14)
    save_plot(fig, filename)
    print(f"Saved: {filename}")

# --- NEW PLOT: Prevalence of Toxic Subtypes within Toxic Comments ---
def plot_toxic_subtype_prevalence_in_toxic_comments(train_df, label_cols, filename="toxic_subtype_prevalence.png"):
    fig, ax = plt.subplots(figsize=(10, 7))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')

    toxic_df = train_df[train_df['is_toxic'] == 1].copy()
    if toxic_df.empty:
        print("No toxic comments found to plot subtypes.")
        return

    # Calculate percentage of toxic comments that have each specific label
    toxic_subtype_percentages = {}
    for col in label_cols:
        count = toxic_df[col].sum()
        percentage = (count / len(toxic_df)) * 100
        toxic_subtype_percentages[col] = percentage

    subtype_df = pd.DataFrame(toxic_subtype_percentages.items(), columns=['Label', 'Percentage'])
    subtype_df = subtype_df.sort_values(by='Percentage', ascending=False)

    sns.barplot(x='Percentage', y='Label', data=subtype_df, palette='viridis', ax=ax)
    ax.set_title('Prevalence of Toxic Subtypes within TOXIC Comments\n(Percentages may sum > 100% due to Multi-labeling)', fontweight='bold', fontsize=16)
    ax.set_xlabel('Percentage of Toxic Comments with this Label', fontsize=14)
    ax.set_ylabel('Toxic Subtype', fontsize=14)
    ax.tick_params(axis='both', labelsize=12)

    # Add text labels
    for index, row in subtype_df.iterrows():
        ax.text(row['Percentage'] + 1, index, f'{row["Percentage"]:.1f}%', va='center', fontsize=10)

    # Add a clarifying note within the plot area
    ax.text(0.95, 0.05,
            "Note: A single toxic comment can have multiple labels.\nThese percentages are relative to the total number of toxic comments.",
            transform=ax.transAxes,
            fontsize=10, verticalalignment='bottom', horizontalalignment='right',
            bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5, ec='k', lw=0.5))

    save_plot(fig, filename)
    print(f"Saved: {filename}")


# --- Call individual plot functions ---
# Uncomment the specific plot functions you want to generate.
# Each plot will be saved as a PNG file in your current working directory.

# Plotting key distributions and comparisons:
plot_overall_toxicity_distribution(train_df)
plot_toxic_subtype_prevalence_in_toxic_comments(train_df, label_cols) # NEW plot with clarity
plot_comment_length_distribution(train_df)
plot_toxic_vs_clean_length(train_df)
plot_label_correlation_heatmap(label_corr)

print("\n--- Plot Description: Label Correlation Heatmap ---")
print("""
This heatmap visualizes the Pearson correlation coefficients between different toxicity labels.
A higher positive value (closer to 1) indicates that two labels frequently co-occur in the same comments,
while values closer to 0 suggest a weaker relationship.

Key observations from this matrix include:
- Strong Positive Correlations: 'Obscene' and 'Insult' exhibit the highest correlation (0.741), meaning comments containing insults are very often also obscene, and vice-versa. 'Toxic' also shows high correlations with 'Obscene' (0.677) and 'Insult' (0.648), indicating these categories frequently overlap.
- Moderate Correlations: 'Severe Toxic' has moderate positive correlations with 'Obscene' (0.490) and 'Insult' (0.487), suggesting that severely toxic comments often contain elements of obscenity or insults.
- Weak/Low Correlations: 'Threat' consistently shows very low correlations with other toxicity types, including 'Severe Toxic' and 'Identity Hate'. This indicates that comments flagged as threats tend to be distinct and do not frequently overlap with other forms of toxicity. Similarly, 'Identity Hate' generally exhibits lower correlations with most other categories, suggesting it often appears independently or with less co-occurrence compared to the more intertwined 'Toxic', 'Obscene', and 'Insult' labels.
""")

plot_multi_label_distribution(train_df, label_cols)
plot_avg_length_per_toxic_label(train_df, label_cols)
plot_text_characteristics_comparison(toxic_patterns, clean_patterns)


print("\nTo generate individual plots, uncomment the desired function calls in the 'Call individual plot functions' section.")
print("Each plot will be saved as a PNG file in your current working directory.")

# 8. SUMMARY INSIGHTS
print("\n" + "=" * 40)
print("8. KEY INSIGHTS SUMMARY")
print("=" * 40)
print("=" * 60)
print("EXPLORATION COMPLETE!")
print("=" * 60)


Train DataFrame shape: (159571, 8)
Test DataFrame shape: (153164, 2)
Test Labels DataFrame shape: (153164, 7)
Sample Submission DataFrame shape: (153164, 7)

TOXIC COMMENT CLASSIFICATION - DATASET EXPLORATION

1. BASIC DATASET INFORMATION
Dataset shape: (159571, 8)
Columns: ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
Memory usage: 97.29 MB

Data types:
id               object
comment_text     object
toxic             int64
severe_toxic      int64
obscene           int64
threat            int64
insult            int64
identity_hate     int64
dtype: object

Missing values:
No missing values found!

2. SAMPLE DATA EXAMINATION

First 3 rows:

Row 0:
Comment: Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't ...
Labels: None (clean)

Row 1:
Comment: D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11...
Labels: None (clean)

Row 2:
Comment: H

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import random # Import random for set_seed
import re # Import re for text processing in EDA (if you combine)
import warnings
import matplotlib.pyplot as plt # Import matplotlib for plotting
import sqlite3 # Added for database integrity checks
import json # For handling JSON data

# For Hugging Face Transformers
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)

# For scikit-learn utilities
from sklearn.model_selection import KFold, train_test_split # Use KFold for CV, train_test_split for main split
from sklearn.metrics import (
    f1_score, precision_score, recall_score, roc_auc_score,
    confusion_matrix, precision_recall_curve, roc_curve, auc,
    average_precision_score # For PR-AUC
)

# For Optuna hyperparameter optimization
import optuna
from optuna.pruners import MedianPruner, HyperbandPruner
from optuna.samplers import TPESampler # Explicitly import TPESampler

# For Imbalance Handling (SMOTE and SMOTETomek)
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek

# For Explainable AI (XAI)
from lime.lime_text import LimeTextExplainer # Import LIME

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# --- IMPORTANT: Explicitly disable Weights & Biases logging entirely ---
os.environ["WANDB_DISABLED"] = "true"

# Configuration Class

class Config:
    # Model and Tokenizer Settings
    MODEL_NAME = "google/electra-base-discriminator"
    MAX_LENGTH = 256  # Maximum sequence length for tokenization

    # Device Configuration
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Use GPU if available, else CPU

    # Reproducibility
    RANDOM_SEED = 42 # Seed for all random number generators to ensure reproducibility

    # Optuna Hyperparameter Optimization Settings
    N_TRIALS = 5 # Number of Optuna trials for hyperparameter optimization
    CV_FOLDS = 5  # Number of folds for cross-validation during Optuna trials
    PATIENCE = 3  # Early stopping patience for the Trainer

    # Label Columns for Multi-label Classification
    LABEL_COLS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    # Original multi-labels (kept for reference, same as LABEL_COLS now)
    ORIGINAL_LABEL_COLS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

    # Optuna Search Spaces for Hyperparameters
    SEARCH_SPACE = {
        'learning_rate': (5e-6, 5e-5), # General range for transformer fine-tuning (log-uniform)
        'batch_size': [16, 32],        # Recommended batch sizes (categorical)
        'num_train_epochs': (2, 5),    # Recommended epoch range for transformers (integer)
        'warmup_ratio': (0.05, 0.2),   # Warmup ratio for learning rate scheduler (float)
        'weight_decay': (0.0, 0.1),    # Regularization to prevent overfitting (float)
        'hidden_dropout_prob': (0.1, 0.3), # Dropout for hidden layers (float)
        'attention_probs_dropout_prob': (0.1, 0.2), # Dropout for attention probabilities (float)
        'gradient_accumulation_steps': [1, 2, 4], # Accumulate gradients to simulate larger batch sizes (categorical)
        'lr_scheduler_type': ['linear', 'cosine'], # Learning rate scheduler types (categorical)
        # New hyperparameter for oversampling ratio (for SMOTE/SMOTETomek)
        'oversampling_ratio': (0.1, 1.0) # From 10% to 100% of minority class size (float)
    }

    # --- Google Drive Save Path Configuration ---
    # IMPORTANT: Ensure your Google Drive is mounted at /content/drive/
    # This is the base path where all models, plots, and CSVs will be saved.
    GOOGLE_DRIVE_SAVE_BASE_PATH = "/content/drive/MyDrive/my_electra_multilabel_classifier_results"

    # --- Optuna Persistent Storage Path ---
    # This is where Optuna's study state will be saved (SQLite database)
    # Using the defined GOOGLE_DRIVE_SAVE_BASE_PATH directly
    OPTUNA_DB_PATH = os.path.join(GOOGLE_DRIVE_SAVE_BASE_PATH, 'optuna_electra_multilabel_study.db')

    # --- New: Option to load existing Optuna study results ---
    # Set to True if you want to skip re-running Optuna optimization
    # and instead load the best parameters from a saved Optuna DB.
    LOAD_EXISTING_STUDY_RESULTS = True # Changed to True to stop further optimization

# Set seeds for reproducibility across different libraries
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed) # For Python's built-in random module

# Dataset Class
class ToxicDataset(Dataset):
    """
    Enhanced dataset class for multi-label toxic comment classification.
    Supports text augmentation and prepares data for Hugging Face models.
    """
    def __init__(self, texts, labels_df, tokenizer, max_length, augment=False):
        self.texts = texts
        self.labels = labels_df # Now a DataFrame with multiple label columns
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.augment = augment # Flag to enable/disable augmentation

    def __len__(self):
        """Returns the total number of samples in the dataset."""
        return len(self.texts)

    def __getitem__(self, idx):
        """
        Retrieves a sample from the dataset.
        Applies augmentation if enabled and randomly triggered.
        Tokenizes the text and returns a dictionary of input features and a multi-label tensor.
        """
        text = str(self.texts.iloc[idx])

        # Simple augmentation for toxic comments to increase diversity
        if self.augment and np.random.random() < 0.3:
            text = self._augment_text(text)

        # Tokenize the text using the pre-trained tokenizer
        encoding = self.tokenizer(
            text,
            truncation=True,        # Truncate to max_length if longer
            padding='max_length',   # Pad to max_length if shorter
            max_length=self.max_length,
            return_tensors='pt'     # Return PyTorch tensors
        )

        # Return a FloatTensor of shape [num_labels] for multi-label classification
        # Ensure labels are converted to float32
        labels_tensor = torch.FloatTensor(self.labels.iloc[idx].values)

        return {
            'input_ids': encoding['input_ids'].flatten(),      # Flatten input_ids tensor
            'attention_mask': encoding['attention_mask'].flatten(), # Flatten attention_mask tensor
            'labels': labels_tensor # Tensor of shape [num_labels]
        }

    def _augment_text(self, text):
        """
        Applies simple text augmentation techniques.
        Currently implements random case changes for a single word.
        """
        words = text.split()
        if len(words) > 2:
            # Randomly select a word and change its case
            idx = np.random.randint(0, len(words))
            if np.random.random() < 0.5:
                words[idx] = words[idx].upper() # Convert to uppercase
            else:
                words[idx] = words[idx].lower() # Convert to lowercase
        return ' '.join(words)

# AdvancedTrainer Class
class AdvancedTrainer(Trainer):
    """
    Custom Hugging Face Trainer with advanced features:
    - Custom loss function for class weighting (BCEWithLogitsLoss with pos_weight).
    - Stores prediction thresholds for evaluation.
    """
    def __init__(self, pos_weights=None, thresholds=None, **kwargs):
        _pos_weights = pos_weights
        _thresholds = thresholds

        super().__init__(**kwargs)

        # pos_weights should now be a tensor of shape [num_labels]
        self.pos_weights = _pos_weights
        # thresholds will be a list of floats, one for each label
        self.thresholds = _thresholds or [0.5] * len(Config.LABEL_COLS)

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """
        Custom loss computation using BCEWithLogitsLoss for multi-label classification.
        Applies `pos_weight` for class imbalance handling if provided.
        """
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')

        # Logits will be [batch_size, num_labels], labels will be [batch_size, num_labels]
        # No squeezing needed here, as BCEWithLogitsLoss expects this shape for multi-label

        # Explicitly cast to float32 to prevent potential dtype mismatches with CUDA kernels
        if labels.dtype != torch.float32:
            labels = labels.to(torch.float32)
        if logits.dtype != torch.float32:
            logits = logits.to(torch.float32)

        # Use BCEWithLogitsLoss which is numerically stable and handles sigmoid internally.
        # Apply pos_weight for class balancing as recommended.
        if self.pos_weights is not None:
            # pos_weight needs to be a tensor of shape [num_labels]
            loss_fct = nn.BCEWithLogitsLoss(pos_weight=self.pos_weights.to(logits.device))
        else:
            loss_fct = nn.BCEWithLogitsLoss()

        loss = loss_fct(logits, labels)

        # The 'num_items_in_batch' argument is now accepted to align with Hugging Face Trainer API changes.
        # You do not need to explicitly use this argument within your loss calculation for BCEWithLogitsLoss,
        # but its presence in the signature is essential to avoid the TypeError.

        return (loss, outputs) if return_outputs else loss

# MetricsCalculator Class
class MetricsCalculator:
    """
    Handles calculation of various classification metrics and threshold optimization
    for multi-label classification.
    """
    def __init__(self, label_cols):
        self.label_cols = label_cols

    def compute_metrics(self, eval_pred, thresholds=None):
        """
        Computes comprehensive evaluation metrics (F1, Precision, Recall, AUC, PR-AUC)
        for multi-label classification, including per-label, micro, and macro averages.
        """
        predictions, labels = eval_pred

        # Apply sigmoid to logits to get probabilities
        sigmoid_preds = torch.sigmoid(torch.tensor(predictions)).numpy()
        labels = labels.astype(int) # Ensure labels are integers for sklearn metrics

        # Use provided thresholds or default to 0.5 for all labels
        thresholds = thresholds if thresholds is not None else [0.5] * len(self.label_cols)

        metrics = {}
        all_binary_preds = []
        all_y_true = []
        all_y_proba = []

        # Calculate metrics for each label and store for macro/micro averaging
        for i, label_name in enumerate(self.label_cols):
            y_true_label = labels[:, i]
            y_pred_proba_label = sigmoid_preds[:, i]
            threshold_label = thresholds[i]

            binary_preds_label = (y_pred_proba_label > threshold_label).astype(int)

            # Store flattened arrays for micro averaging later
            all_y_true.extend(y_true_label)
            all_binary_preds.extend(binary_preds_label)
            all_y_proba.extend(y_pred_proba_label)

            # Per-label metrics with robust error handling for AUC/PR-AUC
            try:
                f1 = f1_score(y_true_label, binary_preds_label, zero_division=0)
                precision = precision_score(y_true_label, binary_preds_label, zero_division=0)
                recall = recall_score(y_true_label, binary_preds_label, zero_division=0)

                auc_score = 0.0
                if len(np.unique(y_true_label)) > 1: # AUC needs at least two unique classes
                    auc_score = roc_auc_score(y_true_label, y_pred_proba_label)

                avg_precision = 0.0
                if len(np.unique(y_true_label)) > 1: # PR-AUC also needs at least two unique classes
                    avg_precision = average_precision_score(y_true_label, y_pred_proba_label)

                tn, fp, fn, tp = confusion_matrix(y_true_label, binary_preds_label).ravel()
            except Exception as e:
                # Catch any error during per-label calculation and assign defaults
                print(f"Warning: Error computing per-label metrics for '{label_name}': {e}", flush=True)
                f1, precision, recall, auc_score, avg_precision = 0.0, 0.0, 0.0, 0.0, 0.0
                tn, fp, fn, tp = 0, 0, 0, 0

            metrics[f'{label_name}_f1'] = f1
            metrics[f'{label_name}_precision'] = precision
            metrics[f'{label_name}_recall'] = recall
            metrics[f'{label_name}_roc_auc'] = auc_score
            metrics[f'{label_name}_pr_auc'] = avg_precision # Average Precision (PR AUC)
            metrics[f'{label_name}_tn'] = tn
            metrics[f'{label_name}_fp'] = fp
            metrics[f'{label_name}_fn'] = fn
            metrics[f'{label_name}_tp'] = tp

        # Calculate Micro F1/Precision/Recall/AUC
        try:
            metrics['micro_f1'] = f1_score(all_y_true, all_binary_preds, average='micro', zero_division=0)
            metrics['micro_precision'] = precision_score(all_y_true, all_binary_preds, average='micro', zero_division=0)
            metrics['micro_recall'] = recall_score(all_y_true, all_binary_preds, average='micro', zero_division=0)

            # Micro-AUC on flattened arrays
            if len(np.unique(all_y_true)) > 1 and len(np.unique(all_y_proba)) > 1:
                metrics['micro_roc_auc'] = roc_auc_score(all_y_true, all_y_proba)
            else:
                metrics['micro_roc_auc'] = 0.0
        except Exception as e:
            print(f"Warning: Error computing micro metrics: {e}", flush=True)
            metrics['micro_f1'], metrics['micro_precision'], metrics['micro_recall'], metrics['micro_roc_auc'] = 0.0, 0.0, 0.0, 0.0


        # Calculate Macro F1/Precision/Recall/AUC
        binary_preds_all_labels = (sigmoid_preds > np.array(thresholds)).astype(int)

        try:
            metrics['macro_f1'] = f1_score(labels, binary_preds_all_labels, average='macro', zero_division=0)
            metrics['macro_precision'] = precision_score(labels, binary_preds_all_labels, average='macro', zero_division=0)
            metrics['macro_recall'] = recall_score(labels, binary_preds_all_labels, average='macro', zero_division=0)
        except Exception as e:
            print(f"ERROR: Failed to compute overall macro F1/Precision/Recall: {e}", flush=True)
            metrics['macro_f1'], metrics['macro_precision'], metrics['macro_recall'] = 0.0, 0.0, 0.0

        # Macro AUC (average of per-label AUCs)
        metrics['macro_roc_auc'] = np.mean([metrics[f'{label_name}_roc_auc'] for label_name in self.label_cols])

        # Trainer uses this as the primary metric for best model selection.
        # This key must match `metric_for_best_model` in your TrainingArguments.
        # We're setting it to macro_f1 as discussed for Optuna optimization.
        metrics['eval_f1'] = metrics['macro_f1']

        return metrics

    def optimize_thresholds(self, y_true, y_pred_proba, n_trials=100):
        """
        Optimizes classification thresholds for each label in a multi-label setting using Optuna.
        The objective is to maximize the micro F1 score across all labels.
        """
        num_labels = y_true.shape[1]
        best_thresholds = []

        def objective(trial):
            thresholds = []
            for i, label_name in enumerate(self.label_cols):
                # Suggest a threshold for each label
                threshold = trial.suggest_float(f'threshold_{label_name}', 0.1, 0.9)
                thresholds.append(threshold)

            # Binarize predictions using the suggested thresholds
            binary_preds = (y_pred_proba > np.array(thresholds)).astype(int)

            # Calculate micro F1 score across all labels
            micro_f1 = f1_score(y_true.flatten(), binary_preds.flatten(), average='micro', zero_division=0)

            return micro_f1

        # Create an Optuna study to maximize the micro F1 score
        study = optuna.create_study(direction='maximize', study_name='multilabel_threshold_optimization')
        study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

        # Extract the best thresholds found by Optuna
        for i, label_name in enumerate(self.label_cols):
            best_thresholds.append(study.best_params[f'threshold_{label_name}'])

        return best_thresholds, study.best_value # Return as a list and the best score

# ModelOptimizer Class
class ModelOptimizer:
    """
    Main class for orchestrating multi-label model training and hyperparameter optimization
    using Optuna and cross-validation, incorporating imbalance handling.
    """
    def __init__(self, train_df, val_df=None):
        self.train_df = train_df
        self.val_df = val_df
        self.tokenizer = AutoTokenizer.from_pretrained(Config.MODEL_NAME)
        self.metrics_calculator = MetricsCalculator(Config.LABEL_COLS)

        # Calculate positive weights for all labels
        self.pos_weights = self._calculate_pos_weights()

    def _calculate_pos_weights(self):
        """
        Calculates positive weights for each label to be used in BCEWithLogitsLoss.
        Weight_i = Number of negative samples_i / Number of positive samples_i.
        """
        pos_weights = []
        for label_col in Config.LABEL_COLS:
            pos_count = self.train_df[label_col].sum()
            neg_count = len(self.train_df) - pos_count
            # Avoid division by zero, set min pos_count to 1
            pos_weight = neg_count / max(pos_count, 1)
            pos_weights.append(pos_weight)
            print(f"Positive weight for '{label_col}': {pos_weight:.2f}", flush=True)

        return torch.FloatTensor(pos_weights)

    def _create_model(self, trial=None):
        """
        Creates an AutoModelForSequenceClassification model from the pre-trained name.
        Applies trial-specific dropout hyperparameters if an Optuna trial is provided.
        """
        model = AutoModelForSequenceClassification.from_pretrained(
            Config.MODEL_NAME,
            num_labels=len(Config.LABEL_COLS), # Now based on all labels
            problem_type="multi_label_classification", # Correct for multi-label BCEWithLogitsLoss
            use_safetensors=True
        )

        # Apply dropout hyperparameters suggested by Optuna if in a trial
        if trial is not None:
            hidden_dropout = trial.suggest_float('hidden_dropout_prob', *Config.SEARCH_SPACE['hidden_dropout_prob'])
            attention_dropout = trial.suggest_float('attention_probs_dropout_prob', *Config.SEARCH_SPACE['attention_probs_dropout_prob'])
            model.config.hidden_dropout_prob = hidden_dropout
            model.config.attention_probs_dropout_prob = attention_dropout

        return model

    def _apply_sampling(self, X_train, y_train, oversampling_ratio, sampling_strategy='hybrid'):
        """
        Applies synthetic oversampling (SMOTE) or hybrid sampling (SMOTETomek) to the training data.
        Only applies to the training set.
        For text data with imblearn, we oversample the *indices* based on a pseudo-label
        (whether a comment has ANY toxic label).
        """
        # Create a "pseudo-label" for sampling: 1 if any label is positive, 0 otherwise
        y_pseudo = (y_train.sum(axis=1) > 0).astype(int)

        if len(np.unique(y_pseudo)) < 2:
            print("Warning: Only one class present in pseudo-labels for sampling. Skipping oversampling.", flush=True)
            return X_train, y_train

        original_indices = np.arange(len(X_train))
        y_pseudo_for_sampling = pd.Series(y_pseudo, index=original_indices) # Keep original indices

        resampled_indices = original_indices
        if sampling_strategy == 'smote':
            # SMOTE works on features. Here we use indices as a proxy.
            # k_neighbors must be less than the number of samples in the minority class.
            # If your minority class is very small, k_neighbors might need adjustment.
            smote_sampler = SMOTE(random_state=Config.RANDOM_SEED, sampling_strategy=oversampling_ratio, k_neighbors=min(3, y_pseudo_for_sampling.value_counts().min() - 1))
            try:
                resampled_indices, _ = smote_sampler.fit_resample(original_indices.reshape(-1, 1), y_pseudo_for_sampling)
            except ValueError as e:
                print(f"SMOTE failed: {e}. Skipping oversampling.", flush=True)
                return X_train, y_train
            resampled_indices = resampled_indices.flatten()

        elif sampling_strategy == 'hybrid':
            # SMOTETomek also works on features.
            smotetomek_sampler = SMOTETomek(random_state=Config.RANDOM_SEED, sampling_strategy=oversampling_ratio, smote=SMOTE(k_neighbors=min(3, y_pseudo_for_sampling.value_counts().min() - 1)))
            try:
                resampled_indices, _ = smotetomek_sampler.fit_resample(original_indices.reshape(-1, 1), y_pseudo_for_sampling)
            except ValueError as e:
                print(f"SMOTETomek failed: {e}. Skipping oversampling.", flush=True)
                return X_train, y_train
            resampled_indices = resampled_indices.flatten()

        # Reconstruct the resampled DataFrame using the resampled indices
        resampled_X_train = X_train.iloc[resampled_indices].reset_index(drop=True)
        resampled_y_train = y_train.iloc[resampled_indices].reset_index(drop=True)

        return resampled_X_train, resampled_y_train

    def _plot_training_history(self, history, output_dir, title_suffix=""):
        """
        Plots training and evaluation metrics over epochs for multi-label.
        Focuses on loss and the main 'eval_f1' (micro-f1).
        """
        epochs = [entry['epoch'] for entry in history if 'loss' in entry]
        train_losses = [entry['loss'] for entry in history if 'loss' in entry]

        eval_epochs = [entry['epoch'] for entry in history if 'eval_loss' in entry]
        val_losses = [entry['eval_loss'] for entry in history if 'eval_loss' in entry]
        val_micro_f1s = [entry['eval_f1'] for entry in history if 'eval_f1' in entry] # eval_f1 is micro_f1

        os.makedirs(output_dir, exist_ok=True)
        plt.figure(figsize=(10, 5))

        plt.subplot(1, 2, 1)
        plt.plot(epochs, train_losses, label='Training Loss')
        plt.plot(eval_epochs, val_losses, label='Validation Loss')
        plt.title(f'Loss per Epoch {title_suffix}')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)

        plt.subplot(1, 2, 2)
        plt.plot(eval_epochs, val_micro_f1s, label='Validation Micro F1-Score', color='orange')
        plt.title(f'Micro F1-Score per Epoch {title_suffix}')
        plt.xlabel('Epoch')
        plt.ylabel('Micro F1-Score')
        plt.legend()
        plt.grid(True)

        plt.tight_layout()
        plot_filename = os.path.join(output_dir, f'training_metrics_plot_{title_suffix.replace(" ", "_")}.png')
        plt.savefig(plot_filename)
        plt.close()
        print(f"    Saved training history plot to {plot_filename}", flush=True)

    def _plot_multilabel_confusion_matrix(self, y_true, y_pred_binary, label_cols, output_dir, title_suffix=""):
        """
        Plots confusion matrices for each label in a multi-label setting.
        """
        num_labels = len(label_cols)
        # Calculate number of rows needed for subplots
        nrows = int(np.ceil(num_labels / 3))
        fig, axes = plt.subplots(nrows=nrows, ncols=3, figsize=(15, 5 * nrows))
        axes = axes.flatten()

        for i, label_name in enumerate(label_cols):
            ax = axes[i]
            cm = confusion_matrix(y_true[:, i], y_pred_binary[:, i])
            tn, fp, fn, tp = cm.ravel()

            ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
            ax.set_title(f'CM for {label_name} {title_suffix}')
            tick_marks = np.arange(2)
            ax.set_xticks(tick_marks, ['0', '1'])
            ax.set_yticks(tick_marks, ['0', '1'])
            ax.set_xlabel('Predicted')
            ax.set_ylabel('True')

            thresh = cm.max() / 2.
            for x in range(cm.shape[0]):
                for y in range(cm.shape[1]):
                    ax.text(y, x, format(cm[x, y], 'd'),
                            ha="center", va="center",
                            color="white" if cm[x, y] > thresh else "black")
        # Hide unused subplots
        for i in range(num_labels, len(axes)):
            fig.delaxes(axes[i])

        plt.tight_layout()
        plot_filename = os.path.join(output_dir, f'multilabel_confusion_matrices_{title_suffix.replace(" ", "_")}.png')
        plt.savefig(plot_filename)
        plt.close()
        print(f"    Saved multilabel confusion matrix plots to {plot_filename}", flush=True)

    def _plot_precision_recall_curve_multilabel(self, y_true, y_pred_proba, label_cols, output_dir, title_suffix=""):
        """
        Plots Precision-Recall Curves for each label.
        """
        plt.figure(figsize=(10, 8))
        for i, label_name in enumerate(label_cols):
            precision, recall, _ = precision_recall_curve(y_true[:, i], y_pred_proba[:, i])
            pr_auc = average_precision_score(y_true[:, i], y_pred_proba[:, i])
            plt.plot(recall, precision, label=f'{label_name} (PR-AUC = {pr_auc:.2f})')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title(f'Precision-Recall Curves {title_suffix}')
        plt.legend(loc='lower left')
        plt.grid(True)
        plt.tight_layout()
        plot_filename = os.path.join(output_dir, f'multilabel_precision_recall_curves_{title_suffix.replace(" ", "_")}.png')
        plt.savefig(plot_filename)
        plt.close()
        print(f"    Saved Multi-label Precision-Recall Curves to {plot_filename}", flush=True)

    def _plot_roc_curve_multilabel(self, y_true, y_pred_proba, label_cols, output_dir, title_suffix=""):
        """
        Plots ROC Curves for each label.
        """
        plt.figure(figsize=(10, 8))
        for i, label_name in enumerate(label_cols):
            fpr, tpr, _ = roc_curve(y_true[:, i], y_pred_proba[:, i])
            roc_auc = auc(fpr, tpr)
            plt.plot(fpr, tpr, label=f'{label_name} (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'Receiver Operating Characteristic (ROC) Curves {title_suffix}')
        plt.legend(loc="lower right")
        plt.grid(True)
        plt.tight_layout()
        plot_filename = os.path.join(output_dir, f'multilabel_roc_curves_{title_suffix.replace(" ", "_")}.png')
        plt.savefig(plot_filename)
        plt.close()
        print(f"    Saved Multi-label ROC Curves to {plot_filename}", flush=True)

    def _plot_probability_distribution_multilabel(self, y_true, y_pred_proba, label_cols, output_dir, title_suffix=""):
        """
        Plots the distribution of predicted probabilities for each class (true positive/negative split).
        """
        num_labels = len(label_cols)
        nrows = int(np.ceil(num_labels / 2))
        fig, axes = plt.subplots(nrows=nrows, ncols=2, figsize=(12, 4 * nrows))
        axes = axes.flatten()

        for i, label_name in enumerate(label_cols):
            ax = axes[i]
            # Probabilities for true negatives for this label
            ax.hist(y_pred_proba[y_true[:, i] == 0, i], bins=50, alpha=0.7, label=f'{label_name} (True Negative)', color='skyblue')
            # Probabilities for true positives for this label
            ax.hist(y_pred_proba[y_true[:, i] == 1, i], bins=50, alpha=0.7, label=f'{label_name} (True Positive)', color='lightcoral')

            ax.set_xlabel('Predicted Probability')
            ax.set_ylabel('Frequency')
            ax.set_title(f'Prob. Dist. for {label_name} {title_suffix}')
            ax.legend()
            ax.grid(True)
        # Hide unused subplots
        for i in range(num_labels, len(axes)):
            fig.delaxes(axes[i])

        plt.tight_layout()
        plot_filename = os.path.join(output_dir, f'multilabel_probability_distributions_{title_suffix.replace(" ", "_")}.png')
        plt.savefig(plot_filename)
        plt.close()
        print(f"    Saved Multi-label Probability Distributions to {plot_filename}", flush=True)


    def objective(self, trial):
        """
        The main Optuna objective function for hyperparameter optimization for multi-label.
        It performs cross-validation for each trial to get a robust evaluation score.
        """
        # Suggest hyperparameters for the current trial based on the search space
        params = {
            'learning_rate': trial.suggest_loguniform('learning_rate', *Config.SEARCH_SPACE['learning_rate']),
            'per_device_train_batch_size': trial.suggest_categorical('batch_size', Config.SEARCH_SPACE['batch_size']),
            'num_train_epochs': trial.suggest_int('num_train_epochs', *Config.SEARCH_SPACE['num_train_epochs']),
            'warmup_ratio': trial.suggest_float('warmup_ratio', *Config.SEARCH_SPACE['warmup_ratio']),
            'weight_decay': trial.suggest_float('weight_decay', *Config.SEARCH_SPACE['weight_decay']),
            'gradient_accumulation_steps': trial.suggest_categorical('gradient_accumulation_steps', Config.SEARCH_SPACE['gradient_accumulation_steps']),
            'lr_scheduler_type': trial.suggest_categorical('lr_scheduler_type', Config.SEARCH_SPACE['lr_scheduler_type']),
            'oversampling_ratio': trial.suggest_float('oversampling_ratio', *Config.SEARCH_SPACE['oversampling_ratio'])
        }

        # Extract oversampling_ratio as it's a custom parameter, not for TrainingArguments
        oversampling_ratio = params.pop('oversampling_ratio') # Use .pop() to remove it from params

        print(f"\nTrial {trial.number} parameters:", flush=True)
        for key, value in params.items():
            print(f"    {key}: {value}", flush=True)
        print(f"    oversampling_ratio: {oversampling_ratio}", flush=True)

        cv_scores = [] # To store micro-F1 scores from each fold

        # Use KFold for cross-validation on indices.
        kf = KFold(n_splits=Config.CV_FOLDS, shuffle=True, random_state=Config.RANDOM_SEED)

        # Split on the indices of the original dataframe
        for fold, (train_idx, val_idx) in enumerate(kf.split(self.train_df)):
            print(f"\n    Fold {fold + 1}/{Config.CV_FOLDS}", flush=True)

            fold_train_df_original = self.train_df.iloc[train_idx].reset_index(drop=True)
            fold_val_df = self.train_df.iloc[val_idx].reset_index(drop=True)

            # --- Apply Hybrid Sampling only on the training set for this fold ---
            resampled_texts, resampled_labels_df = self._apply_sampling(
                X_train=fold_train_df_original['comment_text'],
                y_train=fold_train_df_original[Config.LABEL_COLS],
                oversampling_ratio=oversampling_ratio, # Use the extracted oversampling_ratio
                sampling_strategy='hybrid' # Using hybrid sampling (SMOTETomek)
            )
            print(f"    Training data after sampling: {len(resampled_texts)} samples", flush=True)

            train_dataset = ToxicDataset(
                resampled_texts, # Use resampled texts
                resampled_labels_df, # Use resampled labels
                self.tokenizer,
                Config.MAX_LENGTH,
                augment=True
            )

            val_dataset = ToxicDataset(
                fold_val_df['comment_text'],
                fold_val_df[Config.LABEL_COLS],
                self.tokenizer,
                Config.MAX_LENGTH
            )

            safety_model = self._create_model(trial)

            output_dir_fold = os.path.join(Config.GOOGLE_DRIVE_SAVE_BASE_PATH, f'results_trial_{trial.number}_fold_{fold}')
            os.makedirs(output_dir_fold, exist_ok=True)

            training_args = TrainingArguments(
                output_dir=output_dir_fold,
                eval_strategy="epoch",
                save_strategy="epoch",
                logging_strategy="epoch",
                per_device_eval_batch_size=params['per_device_train_batch_size'],
                save_total_limit=2,
                load_best_model_at_end=True,
                metric_for_best_model="eval_f1",
                greater_is_better=True,
                report_to="none",
                seed=Config.RANDOM_SEED,
                fp16=torch.cuda.is_available(),
                dataloader_pin_memory=False,
                learning_rate=params['learning_rate'],
                per_device_train_batch_size=params['per_device_train_batch_size'],
                num_train_epochs=params['num_train_epochs'],
                warmup_ratio=params['warmup_ratio'],
                weight_decay=params['weight_decay'],
                gradient_accumulation_steps=params['gradient_accumulation_steps'],
                lr_scheduler_type=params['lr_scheduler_type']
            )

            trainer = AdvancedTrainer(
                model=safety_model,
                args=training_args,
                train_dataset=train_dataset,
                eval_dataset=val_dataset,
                compute_metrics=self.metrics_calculator.compute_metrics,
                pos_weights=self.pos_weights,
                callbacks=[EarlyStoppingCallback(early_stopping_patience=Config.PATIENCE)],
            )

            try:
                trainer.train()
                eval_results = trainer.evaluate()
                # 'eval_f1' is correctly used as the primary metric, which your
                # compute_metrics function should set to be the macro F1 for optimization.
                fold_score = eval_results['eval_f1']

                print(f"    Fold {fold + 1} Results (Optimized Metric: Macro F1): {fold_score:.4f}", flush=True)
                print(f"      Macro F1 (from eval_results): {eval_results.get('eval_macro_f1', fold_score):.4f}", flush=True)
                print(f"      Micro F1: {eval_results.get('eval_micro_f1', 'N/A'):.4f}", flush=True)
                print(f"      Micro Precision: {eval_results.get('eval_micro_precision', 'N/A'):.4f}", flush=True)
                print(f"      Micro Recall: {eval_results.get('eval_micro_recall', 'N/A'):.4f}", flush=True)
                print(f"      Micro ROC-AUC: {eval_results.get('eval_micro_roc_auc', 'N/A'):.4f}", flush=True)

                self._plot_training_history(trainer.state.log_history, output_dir_fold, f"Fold {fold + 1}")

                cv_scores.append(fold_score)
            except Exception as e:
                # Improved error reporting for easier debugging
                print(f"    Fold {fold + 1} failed: {type(e).__name__}: {str(e)}", flush=True)
                cv_scores.append(0.0) # Append 0.0 or handle as per your Optuna study design

        mean_cv_score = np.mean(cv_scores)
        std_cv_score = np.std(cv_scores)

        print(f"\nTrial {trial.number} Results:", flush=True)
        print(f"    Mean CV Micro F1: {mean_cv_score:.4f} ± {std_cv_score:.4f}", flush=True)

        return mean_cv_score

    def run_optimization(self): # Renamed from optimize to run_optimization for clarity
        """
        Runs the Optuna hyperparameter optimization study for multi-label classification.
        Incorporates Hyperband pruning for efficient search.
        """
        print("Starting hyperparameter optimization...", flush=True)
        print(f"Search space: {Config.SEARCH_SPACE}", flush=True)

        # Determine max_resource for Hyperband Pruner based on your Config.N_TRAIN_EPOCHS range
        # Assuming Config.SEARCH_SPACE['num_train_epochs'] is the upper bound of your epochs search space
        max_epochs_for_pruner = Config.SEARCH_SPACE['num_train_epochs'][1]

        # Initialize Hyperband Pruner
        pruner = HyperbandPruner(
            min_resource=1,         # Pruning can start after the first epoch/step
            max_resource=max_epochs_for_pruner, # Max epochs a trial can run, matching your search space
            reduction_factor=3      # Common reduction factor for Hyperband
        )

        # Ensure the directory for the Optuna DB exists
        db_dir = os.path.dirname(Config.OPTUNA_DB_PATH.replace("sqlite:///", ""))
        os.makedirs(db_dir, exist_ok=True)
        print(f"Optuna DB directory ensured: {db_dir}", flush=True)

        study = optuna.create_study(
            direction='maximize',
            study_name='electra_toxic_multilabel_classification',
            storage=f'sqlite:///{Config.OPTUNA_DB_PATH}',
            load_if_exists=True,
            sampler=TPESampler(seed=Config.RANDOM_SEED), # Use TPESampler for efficient search
            pruner=pruner # Use the Hyperband Pruner
        )

        print(f"Running {Config.N_TRIALS} trials with Hyperband Pruner...", flush=True)
        study.optimize(self.objective, n_trials=Config.N_TRIALS, show_progress_bar=True)

        print("\nOptimization completed! ✨", flush=True)
        print(f"Best trial number: {study.best_trial.number}", flush=True)

        # Ensure this matches your metric_for_best_model, which should be 'macro_f1'
        print(f"Best Macro F1: {study.best_value:.4f}", flush=True)

        print("Best parameters found: 🚀", flush=True)
        for key, value in study.best_params.items():
            print(f"   {key}: {value}", flush=True)

        return study

    def get_optuna_summary_data(self, study):
        """
        Collects summary data for all Optuna trials.
        """
        summary_data = {
            "total_trials_completed": len(study.trials),
            "best_trial_number": study.best_trial.number,
            "best_objective_value": study.best_value,
            "best_hyperparameters": study.best_trial.params,
            "trial_history": []
        }

        # Collect summary for each trial
        for trial in study.trials:
            trial_info = {
                "trial_number": trial.number,
                "state": str(trial.state),
                "value": trial.value,
                "params": trial.params,
                "duration_seconds": trial.duration.total_seconds() if trial.duration else None
            }
            summary_data["trial_history"].append(trial_info)

        return summary_data

    def train_final_model(self, best_params, save_path=None):
        """
        Trains the final model using the best hyperparameters found during optimization.
        """
        if save_path is None:
            save_path = os.path.join(Config.GOOGLE_DRIVE_SAVE_BASE_PATH, "best_electra_multilabel_model")

        print("\nTraining final model with best parameters...", flush=True)

        # Apply sampling to the entire training set for the final model
        resampled_texts, resampled_labels_df = self._apply_sampling(
            X_train=self.train_df['comment_text'],
            y_train=self.train_df[Config.LABEL_COLS],
            oversampling_ratio=best_params.get('oversampling_ratio', 0.5), # Use optimized ratio or default
            sampling_strategy='hybrid'
        )
        print(f"Final training data after sampling: {len(resampled_texts)} samples", flush=True)


        train_dataset = ToxicDataset(
            resampled_texts,
            resampled_labels_df,
            self.tokenizer,
            Config.MAX_LENGTH,
            augment=True
        )

        val_dataset = None
        if self.val_df is not None:
            val_dataset = ToxicDataset(
                self.val_df['comment_text'],
                self.val_df[Config.LABEL_COLS], # Pass all labels for validation
                self.tokenizer,
                Config.MAX_LENGTH
            )

        safety_model = self._create_model()
        # Apply dropout parameters from best_params if they exist
        if 'hidden_dropout_prob' in best_params:
            safety_model.config.hidden_dropout_prob = best_params['hidden_dropout_prob']
        if 'attention_probs_dropout_prob' in best_params:
            safety_model.config.attention_probs_dropout_prob = best_params['attention_probs_dropout_prob']

        output_dir_final = os.path.join(Config.GOOGLE_DRIVE_SAVE_BASE_PATH, 'final_electra_multilabel_model_results')
        os.makedirs(output_dir_final, exist_ok=True)

        # Prepare TrainingArguments for the final training run
        training_args = TrainingArguments(
            output_dir=output_dir_final,
            eval_strategy="epoch" if val_dataset else "no",
            save_strategy="epoch",
            logging_strategy="epoch",
            per_device_train_batch_size=best_params.get('batch_size', 16),
            per_device_eval_batch_size=best_params.get('batch_size', 16),
            num_train_epochs=best_params.get('num_train_epochs', 3),
            learning_rate=best_params.get('learning_rate', 2e-5),
            warmup_ratio=best_params.get('warmup_ratio', 0.1),
            weight_decay=best_params.get('weight_decay', 0.01),
            gradient_accumulation_steps=best_params.get('gradient_accumulation_steps', 1),
            lr_scheduler_type=best_params.get('lr_scheduler_type', 'linear'),
            save_total_limit=3, # Keep up to 3 best checkpoints
            load_best_model_at_end=True if val_dataset else False,
            metric_for_best_model="eval_f1" if val_dataset else None, # Micro F1 for multilabel
            greater_is_better=True,
            report_to="none",
            seed=Config.RANDOM_SEED,
            fp16=torch.cuda.is_available(),
            dataloader_pin_memory=False,
        )

        trainer = AdvancedTrainer(
            model=safety_model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=self.metrics_calculator.compute_metrics,
            pos_weights=self.pos_weights, # Pass tensor of pos_weights
            callbacks=[EarlyStoppingCallback(early_stopping_patience=Config.PATIENCE)] if val_dataset else [],
        )

        trainer.train()

        if val_dataset:
            final_eval_results = trainer.evaluate()
            print("\nFinal Model Evaluation Results:", flush=True)
            for k, v in final_eval_results.items():
                if k.startswith('eval_'): # Print metrics that were evaluated
                    print(f"    {k.replace('eval_', '')}: {v:.4f}", flush=True)
                elif k.endswith('_f1') or k.endswith('_precision') or k.endswith('_recall') or k.endswith('_roc_auc') or k.endswith('_pr_auc'):
                    print(f"    {k}: {v:.4f}", flush=True)


            # Get true labels and predicted probabilities from the final evaluation
            predictions = trainer.predict(val_dataset)
            y_true_final = predictions.label_ids
            y_pred_proba_final = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()

            # Optimize thresholds on the final validation set for comprehensive evaluation
            print("\nOptimizing classification thresholds for final model evaluation...", flush=True)
            best_thresholds_final, best_threshold_score_final = self.metrics_calculator.optimize_thresholds(
                y_true_final, y_pred_proba_final, n_trials=100
            )
            print(f"Optimized micro F1 (threshold tuning): {best_threshold_score_final:.4f}", flush=True)
            print("Optimized thresholds:", flush=True)
            for i, label_name in enumerate(Config.LABEL_COLS):
                print(f"    {best_thresholds_final[i]:.3f}", flush=True) # Corrected variable name

            # Re-evaluate with optimized thresholds to get final confusion matrices and plots
            final_binary_preds = (y_pred_proba_final > np.array(best_thresholds_final)).astype(int)

            self._plot_training_history(trainer.state.log_history, output_dir_final, "Final Model")
            self._plot_multilabel_confusion_matrix(y_true_final, final_binary_preds, Config.LABEL_COLS, output_dir_final, "Final Model")
            self._plot_precision_recall_curve_multilabel(y_true_final, y_pred_proba_final, Config.LABEL_COLS, output_dir_final, "Final Model")
            self._plot_roc_curve_multilabel(y_true_final, y_pred_proba_final, Config.LABEL_COLS, output_dir_final, "Final Model")
            self._plot_probability_distribution_multilabel(y_true_final, y_pred_proba_final, Config.LABEL_COLS, output_dir_final, "Final Model")

            # Save the optimized thresholds to a CSV file
            threshold_df = pd.DataFrame({
                'label': Config.LABEL_COLS,
                'threshold': best_thresholds_final
            })
            threshold_df.to_csv(os.path.join(Config.GOOGLE_DRIVE_SAVE_BASE_PATH, 'optimized_thresholds_multilabel.csv'), index=False)
            print(f"Final optimized thresholds saved to {os.path.join(Config.GOOGLE_DRIVE_SAVE_BASE_PATH, 'optimized_thresholds_multilabel.csv')}", flush=True)


        trainer.save_model(save_path)
        self.tokenizer.save_pretrained(save_path)

        print(f"Final model saved to {save_path}", flush=True)

        return trainer


# Helper function for LIME predictions (needs to be outside classes to be easily callable by LIME)
def predict_proba_for_lime(texts, model, tokenizer, max_length, device):
    """
    Predictor function required by LIME. Takes a list of raw texts and returns
    prediction probabilities for each class.
    """
    inputs = tokenizer(
        texts,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'
    ).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.sigmoid(outputs.logits).cpu().numpy()
    return probabilities

def main():
    set_seed(Config.RANDOM_SEED) # Ensure reproducibility

    # Create the base directory for saving results
    os.makedirs(Config.GOOGLE_DRIVE_SAVE_BASE_PATH, exist_ok=True)
    print(f"Google Drive base save path ensured: {Config.GOOGLE_DRIVE_SAVE_BASE_PATH}", flush=True)

    print("Loading data...", flush=True)
    try:
        # Load your datasets as specified
        train_df = pd.read_csv("train.csv")
        test_df = pd.read_csv("test.csv")
        test_labels = pd.read_csv("test_labels.csv")
        sample_submission = pd.read_csv("sample_submission.csv")

        print("Data loaded successfully.", flush=True)
        print(f"Train DataFrame shape: {train_df.shape}", flush=True)
        print(f"Test DataFrame shape: {test_df.shape}", flush=True)
        print(f"Test Labels DataFrame shape: {test_labels.shape}", flush=True)
        print(f"Sample Submission DataFrame shape: {sample_submission.shape}", flush=True)


    except FileNotFoundError:
        print("Error: One or more dataset files (train.csv, test.csv, test_labels.csv, sample_submission.csv) not found.", flush=True)
        print("Please ensure all required CSV files are in the same directory as the script.", flush=True)
        return

    # Fill NaN values in 'comment_text'
    train_df['comment_text'] = train_df['comment_text'].fillna('')
    test_df['comment_text'] = test_df['comment_text'].fillna('') # Also fill for test_df if needed

    # --- Use all original labels directly for multilabel classification ---
    # Ensure all label columns are present
    if not all(col in train_df.columns for col in Config.LABEL_COLS):
        print(f"Error: One or more label columns {Config.LABEL_COLS} not found in the training dataset. Exiting.", flush=True)
        return

    print(f"\nLabel columns being used for training: {Config.LABEL_COLS}", flush=True)

    # Display class distribution for each label (important for multilabel)
    print("\nMultilabel Class distribution (Positive count and percentage in training data):", flush=True)
    for col in Config.LABEL_COLS:
        pos_count = train_df[col].sum()
        total_count = len(train_df)
        pos_pct = (pos_count / total_count) * 100
        print(f"  '{col}': {pos_count:,} positive samples ({pos_pct:.2f}%)", flush=True)

    # Splitting the main training data into a training and validation set
    # This validation set is used for the *final model training* and evaluation,
    # distinct from the CV folds within Optuna.
    print("\nSplitting training data into train and validation sets (80/20 split)...", flush=True)
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        train_df['comment_text'],
        train_df[Config.LABEL_COLS],
        test_size=0.2,
        random_state=Config.RANDOM_SEED,
        stratify=train_df['toxic'] # Stratify by 'toxic' label for a more balanced split
    )

    # Reconstruct dataframes for ModelOptimizer
    train_df_for_optimizer = pd.DataFrame({'comment_text': train_texts}).reset_index(drop=True)
    train_labels_for_optimizer = train_labels.reset_index(drop=True)
    val_df_for_optimizer = pd.DataFrame({'comment_text': val_texts}).reset_index(drop=True)
    val_labels_for_optimizer = val_labels.reset_index(drop=True)

    # Merge texts and labels back for convenience in ModelOptimizer initialization
    # (ModelOptimizer expects the full train_df with 'comment_text' and all label columns)
    train_combined_df = pd.concat([train_df_for_optimizer, train_labels_for_optimizer], axis=1)
    val_combined_df = pd.concat([val_df_for_optimizer, val_labels_for_optimizer], axis=1) # Pass this to ModelOptimizer

    print(f"Training set size for final model: {len(train_combined_df)}", flush=True)
    print(f"Validation set size for final model: {len(val_combined_df)}", flush=True)

    optimizer = ModelOptimizer(train_combined_df, val_combined_df) # Pass val_combined_df here

    best_params = {}
    study = None # Initialize study to None
    if Config.LOAD_EXISTING_STUDY_RESULTS:
        print(f"\nAttempting to load existing Optuna study from {Config.OPTUNA_DB_PATH}...", flush=True)
        try:
            study = optuna.load_study(
                study_name='electra_toxic_multilabel_classification',
                storage=f'sqlite:///{Config.OPTUNA_DB_PATH}'
            )
            best_params = study.best_trial.params
            print("Successfully loaded best parameters from existing study:", flush=True)
            for k, v in best_params.items():
                print(f"    {k}: {v}", flush=True)
        except Exception as e:
            print(f"Could not load existing study: {e}. Starting new optimization.", flush=True)
            Config.LOAD_EXISTING_STUDY_RESULTS = False # Fallback to running optimization

    if not Config.LOAD_EXISTING_STUDY_RESULTS:
        study = optimizer.run_optimization() # Changed from .optimize() to .run_optimization()
        best_params = study.best_trial.params

    # Train the final model with the best parameters
    final_trainer = optimizer.train_final_model(best_params)

    print("\nTraining and optimization complete! ", flush=True)
    print(f"Results, model checkpoints, and plots are saved in: {Config.GOOGLE_DRIVE_SAVE_BASE_PATH}", flush=True)

    # --- Generate Optuna Summary Data ---
    if study: # Only generate summary if a study object is available
        optuna_summary = optimizer.get_optuna_summary_data(study)
        print("\n--- Optuna Optimization Summary Data ---", flush=True)
        print(json.dumps(optuna_summary, indent=2), flush=True)
        print("\n--- End of Optuna Optimization Summary Data ---", flush=True)
    else:
        print("\nSkipping Optuna summary data generation as no study object was available.", flush=True)


    # --- Explainability (XAI) Section using LIME ---
    print("\n--- Starting Explainable AI (XAI) Analysis with LIME ---", flush=True)

    # Load the best trained model and tokenizer for XAI
    final_model_path = os.path.join(Config.GOOGLE_DRIVE_SAVE_BASE_PATH, "best_electra_multilabel_model")
    try:
        final_model = AutoModelForSequenceClassification.from_pretrained(final_model_path).to(Config.DEVICE)
        final_tokenizer = AutoTokenizer.from_pretrained(final_model_path)
        print(f"Loaded final model and tokenizer from {final_model_path}", flush=True)
    except Exception as e:
        print(f"Error loading final model for XAI: {e}. Skipping XAI analysis.", flush=True)
        final_model = None # Set to None to prevent further errors if loading failed

    if final_model:
        # Prepare the predictor function for LIME
        lime_predictor = lambda texts: predict_proba_for_lime(
            texts, final_model, final_tokenizer, Config.MAX_LENGTH, Config.DEVICE
        )

        # Initialize LIME Explainer
        explainer = LimeTextExplainer(class_names=Config.LABEL_COLS)

        # Select a few examples for explanation from the validation set
        # You can adjust the number of examples or criteria for selection
        num_xai_examples = min(5, len(val_combined_df)) # Explain up to 5 examples from validation set
        xai_examples_df = val_combined_df.sample(n=num_xai_examples, random_state=Config.RANDOM_SEED).reset_index(drop=True)

        for idx in range(num_xai_examples):
            example_comment_text = xai_examples_df['comment_text'].iloc[idx]
            true_labels = xai_examples_df[Config.LABEL_COLS].iloc[idx].values

            # Get model's raw probabilities for this example
            model_probabilities = lime_predictor([example_comment_text])[0]

            print(f"\n--- XAI Example {idx + 1} ---", flush=True)
            print(f"Comment: '{example_comment_text}'", flush=True)
            print(f"True Labels: {true_labels}", flush=True)
            print(f"Model Probabilities: {model_probabilities.round(3)}", flush=True)

            # Generate explanation for each relevant label
            # You might choose to explain labels where true_label is 1 OR where model_probability > 0.5
            for i, label_name in enumerate(Config.LABEL_COLS):
                # Decide which labels to explain: e.g., true positive, or high predicted probability
                if true_labels[i] == 1 or model_probabilities[i] > 0.5:
                    print(f"\n  Explanation for label: '{label_name}' (True: {true_labels[i]}, Predicted Prob: {model_probabilities[i]:.3f})", flush=True)
                    try:
                        # num_features: how many words to highlight
                        # num_samples: how many perturbed samples LIME generates
                        exp = explainer.explain_instance(
                            example_comment_text,
                            lime_predictor,
                            top_labels=1, # Only get explanation for the top predicted label (within LIME's internal logic)
                            labels=[i],  # Get explanation specifically for this label's index
                            num_features=10, # Number of words to highlight
                            num_samples=1000 # Number of perturbed samples for LIME to generate
                        )
                        # Print explanation as a list of (word, importance_score) tuples
                        print(exp.as_list(label=i), flush=True)
                    except Exception as e:
                        print(f"  Error generating LIME explanation for label '{label_name}': {e}", flush=True)
                else:
                    print(f"  Skipping explanation for label '{label_name}' (True: {true_labels[i]}, Predicted Prob: {model_probabilities[i]:.3f} - not a strong prediction or true positive)", flush=True)

    print("\n--- XAI Analysis Complete ---", flush=True)

if __name__ == '__main__':
    # Mount Google Drive if running in Colab
    # from google.colab import drive
    # drive.mount('/content/drive')
    # print("\nGoogle Drive mounted!", flush=True)

    main()


Google Drive base save path ensured: /content/drive/MyDrive/my_electra_multilabel_classifier_results
Loading data...
Data loaded successfully.
Train DataFrame shape: (159571, 8)
Test DataFrame shape: (153164, 2)
Test Labels DataFrame shape: (153164, 7)
Sample Submission DataFrame shape: (153164, 7)

Label columns being used for training: ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

Multilabel Class distribution (Positive count and percentage in training data):
  'toxic': 15,294 positive samples (9.58%)
  'severe_toxic': 1,595 positive samples (1.00%)
  'obscene': 8,449 positive samples (5.29%)
  'threat': 478 positive samples (0.30%)
  'insult': 7,877 positive samples (4.94%)
  'identity_hate': 1,405 positive samples (0.88%)

Splitting training data into train and validation sets (80/20 split)...
Training set size for final model: 127656
Validation set size for final model: 31915
Positive weight for 'toxic': 9.43
Positive weight for 'severe_toxic': 98.42
P

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1,Toxic F1,Toxic Precision,Toxic Recall,Toxic Roc Auc,Toxic Pr Auc,Toxic Tn,Toxic Fp,Toxic Fn,Toxic Tp,Severe Toxic F1,Severe Toxic Precision,Severe Toxic Recall,Severe Toxic Roc Auc,Severe Toxic Pr Auc,Severe Toxic Tn,Severe Toxic Fp,Severe Toxic Fn,Severe Toxic Tp,Obscene F1,Obscene Precision,Obscene Recall,Obscene Roc Auc,Obscene Pr Auc,Obscene Tn,Obscene Fp,Obscene Fn,Obscene Tp,Threat F1,Threat Precision,Threat Recall,Threat Roc Auc,Threat Pr Auc,Threat Tn,Threat Fp,Threat Fn,Threat Tp,Insult F1,Insult Precision,Insult Recall,Insult Roc Auc,Insult Pr Auc,Insult Tn,Insult Fp,Insult Fn,Insult Tp,Identity Hate F1,Identity Hate Precision,Identity Hate Recall,Identity Hate Roc Auc,Identity Hate Pr Auc,Identity Hate Tn,Identity Hate Fp,Identity Hate Fn,Identity Hate Tp,Micro F1,Micro Precision,Micro Recall,Micro Roc Auc,Macro F1,Macro Precision,Macro Recall,Macro Roc Auc
1,0.608100,0.590379,0.645714,0.808664,0.724263,0.915332,0.987858,0.914454,27790,1066,259,2800,0.452830,0.320427,0.771704,0.988627,0.422795,31095,509,71,240,0.773661,0.648595,0.958480,0.993527,0.900544,29317,888,71,1639,0.578947,0.503817,0.680412,0.977405,0.464883,31753,65,31,66,0.731134,0.606551,0.920126,0.988642,0.811740,29376,949,127,1463,0.529048,0.411538,0.740484,0.987191,0.574177,31320,306,75,214,0.976934,0.976934,0.976934,0.990926,0.645714,0.535865,0.831090,0.987209
2,0.302900,0.627065,0.647134,0.793801,0.692739,0.929389,0.986835,0.909787,27595,1261,216,2843,0.477273,0.351598,0.742765,0.989455,0.449779,31178,426,80,231,0.811999,0.727863,0.918129,0.993558,0.895570,29618,587,140,1570,0.545455,0.442308,0.711340,0.979765,0.483205,31731,87,28,69,0.741230,0.622014,0.916981,0.988653,0.799376,29439,886,132,1458,0.513043,0.374010,0.816609,0.988585,0.572882,31231,395,53,236,0.977592,0.977592,0.977592,0.991236,0.647134,0.535088,0.839202,0.987809
3,0.205500,0.887501,0.663888,0.792669,0.692834,0.926120,0.984219,0.886975,27600,1256,226,2833,0.493082,0.404959,0.630225,0.989108,0.441843,31316,288,115,196,0.821238,0.755152,0.900000,0.992221,0.891017,29706,499,171,1539,0.546154,0.435583,0.731959,0.980264,0.470178,31726,92,26,71,0.757559,0.663671,0.882390,0.987728,0.792052,29614,711,187,1403,0.572626,0.480094,0.709343,0.985552,0.562001,31404,222,84,205,0.979754,0.979754,0.979754,0.990480,0.663888,0.572049,0.796673,0.986515
4,0.145100,1.006753,0.667716,0.816482,0.738177,0.913370,0.985344,0.900385,27865,991,265,2794,0.483173,0.385797,0.646302,0.988721,0.421466,31284,320,110,201,0.822872,0.754634,0.904678,0.991831,0.886228,29702,503,163,1547,0.541833,0.441558,0.701031,0.978552,0.455903,31732,86,29,68,0.759804,0.670029,0.877358,0.987401,0.776451,29638,687,195,1395,0.582133,0.498765,0.698962,0.982079,0.561049,31423,203,87,202,0.980996,0.980996,0.980996,0.990346,0.667716,0.581493,0.790284,0.985655



Final Model Evaluation Results:
    f1: 0.6677
    loss: 1.0068
    toxic_f1: 0.8165
    toxic_precision: 0.7382
    toxic_recall: 0.9134
    toxic_roc_auc: 0.9853
    toxic_pr_auc: 0.9004
    toxic_tn: 27865.0000
    toxic_fp: 991.0000
    toxic_fn: 265.0000
    toxic_tp: 2794.0000
    severe_toxic_f1: 0.4832
    severe_toxic_precision: 0.3858
    severe_toxic_recall: 0.6463
    severe_toxic_roc_auc: 0.9887
    severe_toxic_pr_auc: 0.4215
    severe_toxic_tn: 31284.0000
    severe_toxic_fp: 320.0000
    severe_toxic_fn: 110.0000
    severe_toxic_tp: 201.0000
    obscene_f1: 0.8229
    obscene_precision: 0.7546
    obscene_recall: 0.9047
    obscene_roc_auc: 0.9918
    obscene_pr_auc: 0.8862
    obscene_tn: 29702.0000
    obscene_fp: 503.0000
    obscene_fn: 163.0000
    obscene_tp: 1547.0000
    threat_f1: 0.5418
    threat_precision: 0.4416
    threat_recall: 0.7010
    threat_roc_auc: 0.9786
    threat_pr_auc: 0.4559
    threat_tn: 31732.0000
    threat_fp: 86.0000
    threat_fn: 2

[I 2025-07-09 19:32:39,679] A new study created in memory with name: multilabel_threshold_optimization


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-07-09 19:32:39,703] Trial 0 finished with value: 0.9805681758838581 and parameters: {'threshold_toxic': 0.22323583077071776, 'threshold_severe_toxic': 0.1733485978739659, 'threshold_obscene': 0.6670869540338911, 'threshold_threat': 0.5189033379274536, 'threshold_insult': 0.8422024163010693, 'threshold_identity_hate': 0.45258171007393855}. Best is trial 0 with value: 0.9805681758838581.
[I 2025-07-09 19:32:39,716] Trial 1 finished with value: 0.9804428429683012 and parameters: {'threshold_toxic': 0.5116042770514322, 'threshold_severe_toxic': 0.47801045732770886, 'threshold_obscene': 0.2745389364212583, 'threshold_threat': 0.13796264095507854, 'threshold_insult': 0.179759147383185, 'threshold_identity_hate': 0.6814491967306048}. Best is trial 0 with value: 0.9805681758838581.
[I 2025-07-09 19:32:39,730] Trial 2 finished with value: 0.9814872839312758 and parameters: {'threshold_toxic': 0.7522295340246085, 'threshold_severe_toxic': 0.8815877199980144, 'threshold_obscene': 0.407004

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import random # Import random for set_seed
import re # Import re for text processing in EDA (if you combine)
import warnings
import matplotlib.pyplot as plt # Import matplotlib for plotting
import sqlite3 # Added for database integrity checks
import json # For handling JSON data

# For Hugging Face Transformers
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)

# For scikit-learn utilities
from sklearn.model_selection import KFold, train_test_split # Use KFold for CV, train_test_split for main split
from sklearn.metrics import (
    f1_score, precision_score, recall_score, roc_auc_score,
    confusion_matrix, precision_recall_curve, roc_curve, auc,
    average_precision_score # For PR-AUC
)

# For Optuna hyperparameter optimization
import optuna
from optuna.pruners import MedianPruner, HyperbandPruner
from optuna.samplers import TPESampler # Explicitly import TPESampler

# For Imbalance Handling (SMOTE and SMOTETomek)
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek

# For Explainable AI (XAI)
from lime.lime_text import LimeTextExplainer # Import LIME

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# --- IMPORTANT: Explicitly disable Weights & Biases logging entirely ---
os.environ["WANDB_DISABLED"] = "true"

# Configuration Class

class Config:
    # Model and Tokenizer Settings
    MODEL_NAME = "google/electra-base-discriminator"
    MAX_LENGTH = 256  # Maximum sequence length for tokenization

    # Device Configuration
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Use GPU if available, else CPU

    # Reproducibility
    RANDOM_SEED = 42 # Seed for all random number generators to ensure reproducibility

    # Optuna Hyperparameter Optimization Settings
    N_TRIALS = 5 # Number of Optuna trials for hyperparameter optimization
    CV_FOLDS = 5  # Number of folds for cross-validation during Optuna trials
    PATIENCE = 3  # Early stopping patience for the Trainer

    # Label Columns for Multi-label Classification
    LABEL_COLS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    # Original multi-labels (kept for reference, same as LABEL_COLS now)
    ORIGINAL_LABEL_COLS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

    # Optuna Search Spaces for Hyperparameters
    SEARCH_SPACE = {
        'learning_rate': (5e-6, 5e-5), # General range for transformer fine-tuning (log-uniform)
        'batch_size': [16, 32],        # Recommended batch sizes (categorical)
        'num_train_epochs': (2, 5),    # Recommended epoch range for transformers (integer)
        'warmup_ratio': (0.05, 0.2),   # Warmup ratio for learning rate scheduler (float)
        'weight_decay': (0.0, 0.1),    # Regularization to prevent overfitting (float)
        'hidden_dropout_prob': (0.1, 0.3), # Dropout for hidden layers (float)
        'attention_probs_dropout_prob': (0.1, 0.2), # Dropout for attention probabilities (float)
        'gradient_accumulation_steps': [1, 2, 4], # Accumulate gradients to simulate larger batch sizes (categorical)
        'lr_scheduler_type': ['linear', 'cosine'], # Learning rate scheduler types (categorical)
        # New hyperparameter for oversampling ratio (for SMOTE/SMOTETomek)
        'oversampling_ratio': (0.1, 1.0) # From 10% to 100% of minority class size (float)
    }

    # --- Google Drive Save Path Configuration ---
    # IMPORTANT: Ensure your Google Drive is mounted at /content/drive/
    # This is the base path where all models, plots, and CSVs will be saved.
    GOOGLE_DRIVE_SAVE_BASE_PATH = "/content/drive/MyDrive/my_electra_multilabel_classifier_results"

    # --- Optuna Persistent Storage Path ---
    # This is where Optuna's study state will be saved (SQLite database)
    # Using the defined GOOGLE_DRIVE_SAVE_BASE_PATH directly
    OPTUNA_DB_PATH = os.path.join(GOOGLE_DRIVE_SAVE_BASE_PATH, 'optuna_electra_multilabel_study.db')

    # --- New: Option to load existing Optuna study results ---
    # Set to True if you want to skip re-running Optuna optimization
    # and instead load the best parameters from a saved Optuna DB.
    LOAD_EXISTING_STUDY_RESULTS = True # Changed to True to stop further optimization

# Set seeds for reproducibility across different libraries
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed) # For Python's built-in random module

# Dataset Class
class ToxicDataset(Dataset):
    """
    Enhanced dataset class for multi-label toxic comment classification.
    Supports text augmentation and prepares data for Hugging Face models.
    """
    def __init__(self, texts, labels_df, tokenizer, max_length, augment=False):
        self.texts = texts
        self.labels = labels_df # Now a DataFrame with multiple label columns
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.augment = augment # Flag to enable/disable augmentation

    def __len__(self):
        """Returns the total number of samples in the dataset."""
        return len(self.texts)

    def __getitem__(self, idx):
        """
        Retrieves a sample from the dataset.
        Applies augmentation if enabled and randomly triggered.
        Tokenizes the text and returns a dictionary of input features and a multi-label tensor.
        """
        text = str(self.texts.iloc[idx])

        # Simple augmentation for toxic comments to increase diversity
        if self.augment and np.random.random() < 0.3:
            text = self._augment_text(text)

        # Tokenize the text using the pre-trained tokenizer
        encoding = self.tokenizer(
            text,
            truncation=True,        # Truncate to max_length if longer
            padding='max_length',   # Pad to max_length if shorter
            max_length=self.max_length,
            return_tensors='pt'     # Return PyTorch tensors
        )

        # Return a FloatTensor of shape [num_labels] for multi-label classification
        # Ensure labels are converted to float32
        labels_tensor = torch.FloatTensor(self.labels.iloc[idx].values)

        return {
            'input_ids': encoding['input_ids'].flatten(),      # Flatten input_ids tensor
            'attention_mask': encoding['attention_mask'].flatten(), # Flatten attention_mask tensor
            'labels': labels_tensor # Tensor of shape [num_labels]
        }

    def _augment_text(self, text):
        """
        Applies simple text augmentation techniques.
        Currently implements random case changes for a single word.
        """
        words = text.split()
        if len(words) > 2:
            # Randomly select a word and change its case
            idx = np.random.randint(0, len(words))
            if np.random.random() < 0.5:
                words[idx] = words[idx].upper() # Convert to uppercase
            else:
                words[idx] = words[idx].lower() # Convert to lowercase
        return ' '.join(words)

# AdvancedTrainer Class
class AdvancedTrainer(Trainer):
    """
    Custom Hugging Face Trainer with advanced features:
    - Custom loss function for class weighting (BCEWithLogitsLoss with pos_weight).
    - Stores prediction thresholds for evaluation.
    """
    def __init__(self, pos_weights=None, thresholds=None, **kwargs):
        _pos_weights = pos_weights
        _thresholds = thresholds

        super().__init__(**kwargs)

        # pos_weights should now be a tensor of shape [num_labels]
        self.pos_weights = _pos_weights
        # thresholds will be a list of floats, one for each label
        self.thresholds = _thresholds or [0.5] * len(Config.LABEL_COLS)

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """
        Custom loss computation using BCEWithLogitsLoss for multi-label classification.
        Applies `pos_weight` for class imbalance handling if provided.
        """
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')

        # Logits will be [batch_size, num_labels], labels will be [batch_size, num_labels]
        # No squeezing needed here, as BCEWithLogitsLoss expects this shape for multi-label

        # Explicitly cast to float32 to prevent potential dtype mismatches with CUDA kernels
        if labels.dtype != torch.float32:
            labels = labels.to(torch.float32)
        if logits.dtype != torch.float32:
            logits = logits.to(torch.float32)

        # Use BCEWithLogitsLoss which is numerically stable and handles sigmoid internally.
        # Apply pos_weight for class balancing as recommended.
        if self.pos_weights is not None:
            # pos_weight needs to be a tensor of shape [num_labels]
            loss_fct = nn.BCEWithLogitsLoss(pos_weight=self.pos_weights.to(logits.device))
        else:
            loss_fct = nn.BCEWithLogitsLoss()

        loss = loss_fct(logits, labels)

        # The 'num_items_in_batch' argument is now accepted to align with Hugging Face Trainer API changes.
        # You do not need to explicitly use this argument within your loss calculation for BCEWithLogitsLoss,
        # but its presence in the signature is essential to avoid the TypeError.

        return (loss, outputs) if return_outputs else loss

# MetricsCalculator Class
class MetricsCalculator:
    """
    Handles calculation of various classification metrics and threshold optimization
    for multi-label classification.
    """
    def __init__(self, label_cols):
        self.label_cols = label_cols

    def compute_metrics(self, eval_pred, thresholds=None):
        """
        Computes comprehensive evaluation metrics (F1, Precision, Recall, AUC, PR-AUC)
        for multi-label classification, including per-label, micro, and macro averages.
        """
        predictions, labels = eval_pred

        # Apply sigmoid to logits to get probabilities
        sigmoid_preds = torch.sigmoid(torch.tensor(predictions)).numpy()
        labels = labels.astype(int) # Ensure labels are integers for sklearn metrics

        # Use provided thresholds or default to 0.5 for all labels
        thresholds = thresholds if thresholds is not None else [0.5] * len(self.label_cols)

        metrics = {}
        all_binary_preds = []
        all_y_true = []
        all_y_proba = []

        # Calculate metrics for each label and store for macro/micro averaging
        for i, label_name in enumerate(self.label_cols):
            y_true_label = labels[:, i]
            y_pred_proba_label = sigmoid_preds[:, i]
            threshold_label = thresholds[i]

            binary_preds_label = (y_pred_proba_label > threshold_label).astype(int)

            # Store flattened arrays for micro averaging later
            all_y_true.extend(y_true_label)
            all_binary_preds.extend(binary_preds_label)
            all_y_proba.extend(y_pred_proba_label)

            # Per-label metrics with robust error handling for AUC/PR-AUC
            try:
                f1 = f1_score(y_true_label, binary_preds_label, zero_division=0)
                precision = precision_score(y_true_label, binary_preds_label, zero_division=0)
                recall = recall_score(y_true_label, binary_preds_label, zero_division=0)

                auc_score = 0.0
                if len(np.unique(y_true_label)) > 1: # AUC needs at least two unique classes
                    auc_score = roc_auc_score(y_true_label, y_pred_proba_label)

                avg_precision = 0.0
                if len(np.unique(y_true_label)) > 1: # PR-AUC also needs at least two unique classes
                    avg_precision = average_precision_score(y_true_label, y_pred_proba_label)

                tn, fp, fn, tp = confusion_matrix(y_true_label, binary_preds_label).ravel()
            except Exception as e:
                # Catch any error during per-label calculation and assign defaults
                print(f"Warning: Error computing per-label metrics for '{label_name}': {e}", flush=True)
                f1, precision, recall, auc_score, avg_precision = 0.0, 0.0, 0.0, 0.0, 0.0
                tn, fp, fn, tp = 0, 0, 0, 0

            metrics[f'{label_name}_f1'] = f1
            metrics[f'{label_name}_precision'] = precision
            metrics[f'{label_name}_recall'] = recall
            metrics[f'{label_name}_roc_auc'] = auc_score
            metrics[f'{label_name}_pr_auc'] = avg_precision # Average Precision (PR AUC)
            metrics[f'{label_name}_tn'] = tn
            metrics[f'{label_name}_fp'] = fp
            metrics[f'{label_name}_fn'] = fn
            metrics[f'{label_name}_tp'] = tp

        # Calculate Micro F1/Precision/Recall/AUC
        try:
            metrics['micro_f1'] = f1_score(all_y_true, all_binary_preds, average='micro', zero_division=0)
            metrics['micro_precision'] = precision_score(all_y_true, all_binary_preds, average='micro', zero_division=0)
            metrics['micro_recall'] = recall_score(all_y_true, all_binary_preds, average='micro', zero_division=0)

            # Micro-AUC on flattened arrays
            if len(np.unique(all_y_true)) > 1 and len(np.unique(all_y_proba)) > 1:
                metrics['micro_roc_auc'] = roc_auc_score(all_y_true, all_y_proba)
            else:
                metrics['micro_roc_auc'] = 0.0
        except Exception as e:
            print(f"Warning: Error computing micro metrics: {e}", flush=True)
            metrics['micro_f1'], metrics['micro_precision'], metrics['micro_recall'], metrics['micro_roc_auc'] = 0.0, 0.0, 0.0, 0.0


        # Calculate Macro F1/Precision/Recall/AUC
        binary_preds_all_labels = (sigmoid_preds > np.array(thresholds)).astype(int)

        try:
            metrics['macro_f1'] = f1_score(labels, binary_preds_all_labels, average='macro', zero_division=0)
            metrics['macro_precision'] = precision_score(labels, binary_preds_all_labels, average='macro', zero_division=0)
            metrics['macro_recall'] = recall_score(labels, binary_preds_all_labels, average='macro', zero_division=0)
        except Exception as e:
            print(f"ERROR: Failed to compute overall macro F1/Precision/Recall: {e}", flush=True)
            metrics['macro_f1'], metrics['macro_precision'], metrics['macro_recall'] = 0.0, 0.0, 0.0

        # Macro AUC (average of per-label AUCs)
        metrics['macro_roc_auc'] = np.mean([metrics[f'{label_name}_roc_auc'] for label_name in self.label_cols])

        # Trainer uses this as the primary metric for best model selection.
        # This key must match `metric_for_best_model` in your TrainingArguments.
        # We're setting it to macro_f1 as discussed for Optuna optimization.
        metrics['eval_f1'] = metrics['macro_f1']

        return metrics

    def optimize_thresholds(self, y_true, y_pred_proba, n_trials=100):
        """
        Optimizes classification thresholds for each label in a multi-label setting using Optuna.
        The objective is to maximize the macro F1 score across all labels.
        """
        num_labels = y_true.shape[1]
        best_thresholds = []

        def objective(trial):
            thresholds = []
            for i, label_name in enumerate(self.label_cols):
                # Suggest a threshold for each label
                threshold = trial.suggest_float(f'threshold_{label_name}', 0.1, 0.9)
                thresholds.append(threshold)

            # Binarize predictions using the suggested thresholds
            binary_preds = (y_pred_proba > np.array(thresholds)).astype(int)

            # Calculate macro F1 score across all labels (CHANGED TO MACRO F1)
            macro_f1 = f1_score(y_true, binary_preds, average='macro', zero_division=0)

            return macro_f1

        # Create an Optuna study to maximize the macro F1 score
        study = optuna.create_study(direction='maximize', study_name='multilabel_threshold_optimization')
        study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

        # Extract the best thresholds found by Optuna
        for i, label_name in enumerate(self.label_cols):
            best_thresholds.append(study.best_params[f'threshold_{label_name}'])

        return best_thresholds, study.best_value # Return as a list and the best score

# ModelOptimizer Class
class ModelOptimizer:
    """
    Main class for orchestrating multi-label model training and hyperparameter optimization
    using Optuna and cross-validation, incorporating imbalance handling.
    """
    def __init__(self, train_df, val_df=None):
        self.train_df = train_df
        self.val_df = val_df
        self.tokenizer = AutoTokenizer.from_pretrained(Config.MODEL_NAME)
        self.metrics_calculator = MetricsCalculator(Config.LABEL_COLS)

        # Calculate positive weights for all labels
        self.pos_weights = self._calculate_pos_weights()

    def _calculate_pos_weights(self):
        """
        Calculates positive weights for each label to be used in BCEWithLogitsLoss.
        Weight_i = Number of negative samples_i / Number of positive samples_i.
        """
        pos_weights = []
        for label_col in Config.LABEL_COLS:
            pos_count = self.train_df[label_col].sum()
            neg_count = len(self.train_df) - pos_count
            # Avoid division by zero, set min pos_count to 1
            pos_weight = neg_count / max(pos_count, 1)
            pos_weights.append(pos_weight)
            print(f"Positive weight for '{label_col}': {pos_weight:.2f}", flush=True)

        return torch.FloatTensor(pos_weights)

    def _create_model(self, trial=None):
        """
        Creates an AutoModelForSequenceClassification model from the pre-trained name.
        Applies trial-specific dropout hyperparameters if an Optuna trial is provided.
        """
        model = AutoModelForSequenceClassification.from_pretrained(
            Config.MODEL_NAME,
            num_labels=len(Config.LABEL_COLS), # Now based on all labels
            problem_type="multi_label_classification", # Correct for multi-label BCEWithLogitsLoss
            use_safetensors=True
        )

        # Apply dropout hyperparameters suggested by Optuna if in a trial
        if trial is not None:
            hidden_dropout = trial.suggest_float('hidden_dropout_prob', *Config.SEARCH_SPACE['hidden_dropout_prob'])
            attention_dropout = trial.suggest_float('attention_probs_dropout_prob', *Config.SEARCH_SPACE['attention_probs_dropout_prob'])
            model.config.hidden_dropout_prob = hidden_dropout
            model.config.attention_probs_dropout_prob = attention_dropout

        return model

    def _apply_sampling(self, X_train, y_train, oversampling_ratio, sampling_strategy='hybrid'):
        """
        Applies synthetic oversampling (SMOTE) or hybrid sampling (SMOTETomek) to the training data.
        Only applies to the training set.
        For text data with imblearn, we oversample the *indices* based on a pseudo-label
        (whether a comment has ANY toxic label).
        """
        # Create a "pseudo-label" for sampling: 1 if any label is positive, 0 otherwise
        y_pseudo = (y_train.sum(axis=1) > 0).astype(int)

        if len(np.unique(y_pseudo)) < 2:
            print("Warning: Only one class present in pseudo-labels for sampling. Skipping oversampling.", flush=True)
            return X_train, y_train

        original_indices = np.arange(len(X_train))
        y_pseudo_for_sampling = pd.Series(y_pseudo, index=original_indices) # Keep original indices

        resampled_indices = original_indices
        if sampling_strategy == 'smote':
            # SMOTE works on features. Here we use indices as a proxy.
            # k_neighbors must be less than the number of samples in the minority class.
            # If your minority class is very small, k_neighbors might need adjustment.
            smote_sampler = SMOTE(random_state=Config.RANDOM_SEED, sampling_strategy=oversampling_ratio, k_neighbors=min(3, y_pseudo_for_sampling.value_counts().min() - 1))
            try:
                resampled_indices, _ = smote_sampler.fit_resample(original_indices.reshape(-1, 1), y_pseudo_for_sampling)
            except ValueError as e:
                print(f"SMOTE failed: {e}. Skipping oversampling.", flush=True)
                return X_train, y_train
            resampled_indices = resampled_indices.flatten()

        elif sampling_strategy == 'hybrid':
            # SMOTETomek also works on features.
            smotetomek_sampler = SMOTETomek(random_state=Config.RANDOM_SEED, sampling_strategy=oversampling_ratio, smote=SMOTE(k_neighbors=min(3, y_pseudo_for_sampling.value_counts().min() - 1)))
            try:
                resampled_indices, _ = smotetomek_sampler.fit_resample(original_indices.reshape(-1, 1), y_pseudo_for_sampling)
            except ValueError as e:
                print(f"SMOTETomek failed: {e}. Skipping oversampling.", flush=True)
                return X_train, y_train
            resampled_indices = resampled_indices.flatten()

        # Reconstruct the resampled DataFrame using the resampled indices
        resampled_X_train = X_train.iloc[resampled_indices].reset_index(drop=True)
        resampled_y_train = y_train.iloc[resampled_indices].reset_index(drop=True)

        return resampled_X_train, resampled_y_train

    def _plot_training_history(self, history, output_dir, title_suffix=""):
        """
        Plots training and evaluation metrics over epochs for multi-label.
        Focuses on loss and the main 'eval_f1' (micro-f1).
        """
        epochs = [entry['epoch'] for entry in history if 'loss' in entry]
        train_losses = [entry['loss'] for entry in history if 'loss' in entry]

        eval_epochs = [entry['epoch'] for entry in history if 'eval_loss' in entry]
        val_losses = [entry['eval_loss'] for entry in history if 'eval_loss' in entry]
        val_micro_f1s = [entry['eval_f1'] for entry in history if 'eval_f1' in entry] # eval_f1 is micro_f1

        os.makedirs(output_dir, exist_ok=True)
        plt.figure(figsize=(10, 5))

        plt.subplot(1, 2, 1)
        plt.plot(epochs, train_losses, label='Training Loss')
        plt.plot(eval_epochs, val_losses, label='Validation Loss')
        plt.title(f'Loss per Epoch {title_suffix}')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)

        plt.subplot(1, 2, 2)
        plt.plot(eval_epochs, val_micro_f1s, label='Validation Micro F1-Score', color='orange')
        plt.title(f'Micro F1-Score per Epoch {title_suffix}')
        plt.xlabel('Epoch')
        plt.ylabel('Micro F1-Score')
        plt.legend()
        plt.grid(True)

        plt.tight_layout()
        plot_filename = os.path.join(output_dir, f'training_metrics_plot_{title_suffix.replace(" ", "_")}.png')
        plt.savefig(plot_filename)
        plt.close()
        print(f"    Saved training history plot to {plot_filename}", flush=True)

    def _plot_multilabel_confusion_matrix(self, y_true, y_pred_binary, label_cols, output_dir, title_suffix=""):
        """
        Plots confusion matrices for each label in a multi-label setting.
        """
        num_labels = len(label_cols)
        # Calculate number of rows needed for subplots
        nrows = int(np.ceil(num_labels / 3))
        fig, axes = plt.subplots(nrows=nrows, ncols=3, figsize=(15, 5 * nrows))
        axes = axes.flatten()

        for i, label_name in enumerate(label_cols):
            ax = axes[i]
            cm = confusion_matrix(y_true[:, i], y_pred_binary[:, i])
            tn, fp, fn, tp = cm.ravel()

            ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
            ax.set_title(f'CM for {label_name} {title_suffix}')
            tick_marks = np.arange(2)
            ax.set_xticks(tick_marks, ['0', '1'])
            ax.set_yticks(tick_marks, ['0', '1'])
            ax.set_xlabel('Predicted')
            ax.set_ylabel('True')

            thresh = cm.max() / 2.
            for x in range(cm.shape[0]):
                for y in range(cm.shape[1]):
                    ax.text(y, x, format(cm[x, y], 'd'),
                            ha="center", va="center",
                            color="white" if cm[x, y] > thresh else "black")
        # Hide unused subplots
        for i in range(num_labels, len(axes)):
            fig.delaxes(axes[i])

        plt.tight_layout()
        plot_filename = os.path.join(output_dir, f'multilabel_confusion_matrices_{title_suffix.replace(" ", "_")}.png')
        plt.savefig(plot_filename)
        plt.close()
        print(f"    Saved multilabel confusion matrix plots to {plot_filename}", flush=True)

    def _plot_precision_recall_curve_multilabel(self, y_true, y_pred_proba, label_cols, output_dir, title_suffix=""):
        """
        Plots Precision-Recall Curves for each label.
        """
        plt.figure(figsize=(10, 8))
        for i, label_name in enumerate(label_cols):
            precision, recall, _ = precision_recall_curve(y_true[:, i], y_pred_proba[:, i])
            pr_auc = average_precision_score(y_true[:, i], y_pred_proba[:, i])
            plt.plot(recall, precision, label=f'{label_name} (PR-AUC = {pr_auc:.2f})')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title(f'Precision-Recall Curves {title_suffix}')
        plt.legend(loc='lower left')
        plt.grid(True)
        plt.tight_layout()
        plot_filename = os.path.join(output_dir, f'multilabel_precision_recall_curves_{title_suffix.replace(" ", "_")}.png')
        plt.savefig(plot_filename)
        plt.close()
        print(f"    Saved Multi-label Precision-Recall Curves to {plot_filename}", flush=True)

    def _plot_roc_curve_multilabel(self, y_true, y_pred_proba, label_cols, output_dir, title_suffix=""):
        """
        Plots ROC Curves for each label.
        """
        plt.figure(figsize=(10, 8))
        for i, label_name in enumerate(label_cols):
            fpr, tpr, _ = roc_curve(y_true[:, i], y_pred_proba[:, i])
            roc_auc = auc(fpr, tpr)
            plt.plot(fpr, tpr, label=f'{label_name} (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'Receiver Operating Characteristic (ROC) Curves {title_suffix}')
        plt.legend(loc="lower right")
        plt.grid(True)
        plt.tight_layout()
        plot_filename = os.path.join(output_dir, f'multilabel_roc_curves_{title_suffix.replace(" ", "_")}.png')
        plt.savefig(plot_filename)
        plt.close()
        print(f"    Saved Multi-label ROC Curves to {plot_filename}", flush=True)

    def _plot_probability_distribution_multilabel(self, y_true, y_pred_proba, label_cols, output_dir, title_suffix=""):
        """
        Plots the distribution of predicted probabilities for each class (true positive/negative split).
        """
        num_labels = len(label_cols)
        nrows = int(np.ceil(num_labels / 2))
        fig, axes = plt.subplots(nrows=nrows, ncols=2, figsize=(12, 4 * nrows))
        axes = axes.flatten()

        for i, label_name in enumerate(label_cols):
            ax = axes[i]
            # Probabilities for true negatives for this label
            ax.hist(y_pred_proba[y_true[:, i] == 0, i], bins=50, alpha=0.7, label=f'{label_name} (True Negative)', color='skyblue')
            # Probabilities for true positives for this label
            ax.hist(y_pred_proba[y_true[:, i] == 1, i], bins=50, alpha=0.7, label=f'{label_name} (True Positive)', color='lightcoral')

            ax.set_xlabel('Predicted Probability')
            ax.set_ylabel('Frequency')
            ax.set_title(f'Prob. Dist. for {label_name} {title_suffix}')
            ax.legend()
            ax.grid(True)
        # Hide unused subplots
        for i in range(num_labels, len(axes)):
            fig.delaxes(axes[i])

        plt.tight_layout()
        plot_filename = os.path.join(output_dir, f'multilabel_probability_distributions_{title_suffix.replace(" ", "_")}.png')
        plt.savefig(plot_filename)
        plt.close()
        print(f"    Saved Multi-label Probability Distributions to {plot_filename}", flush=True)


    def objective(self, trial):
        """
        The main Optuna objective function for hyperparameter optimization for multi-label.
        It performs cross-validation for each trial to get a robust evaluation score.
        """
        # Suggest hyperparameters for the current trial based on the search space
        params = {
            'learning_rate': trial.suggest_loguniform('learning_rate', *Config.SEARCH_SPACE['learning_rate']),
            'per_device_train_batch_size': trial.suggest_categorical('batch_size', Config.SEARCH_SPACE['batch_size']),
            'num_train_epochs': trial.suggest_int('num_train_epochs', *Config.SEARCH_SPACE['num_train_epochs']),
            'warmup_ratio': trial.suggest_float('warmup_ratio', *Config.SEARCH_SPACE['warmup_ratio']),
            'weight_decay': trial.suggest_float('weight_decay', *Config.SEARCH_SPACE['weight_decay']),
            'gradient_accumulation_steps': trial.suggest_categorical('gradient_accumulation_steps', Config.SEARCH_SPACE['gradient_accumulation_steps']),
            'lr_scheduler_type': trial.suggest_categorical('lr_scheduler_type', Config.SEARCH_SPACE['lr_scheduler_type']),
            'oversampling_ratio': trial.suggest_float('oversampling_ratio', *Config.SEARCH_SPACE['oversampling_ratio'])
        }

        # Extract oversampling_ratio as it's a custom parameter, not for TrainingArguments
        oversampling_ratio = params.pop('oversampling_ratio') # Use .pop() to remove it from params

        print(f"\nTrial {trial.number} parameters:", flush=True)
        for key, value in params.items():
            print(f"    {key}: {value}", flush=True)
        print(f"    oversampling_ratio: {oversampling_ratio}", flush=True)

        cv_scores = [] # To store micro-F1 scores from each fold

        # Use KFold for cross-validation on indices.
        kf = KFold(n_splits=Config.CV_FOLDS, shuffle=True, random_state=Config.RANDOM_SEED)

        # Split on the indices of the original dataframe
        for fold, (train_idx, val_idx) in enumerate(kf.split(self.train_df)):
            print(f"\n    Fold {fold + 1}/{Config.CV_FOLDS}", flush=True)

            fold_train_df_original = self.train_df.iloc[train_idx].reset_index(drop=True)
            fold_val_df = self.train_df.iloc[val_idx].reset_index(drop=True)

            # --- Apply Hybrid Sampling only on the training set for this fold ---
            resampled_texts, resampled_labels_df = self._apply_sampling(
                X_train=fold_train_df_original['comment_text'],
                y_train=fold_train_df_original[Config.LABEL_COLS],
                oversampling_ratio=oversampling_ratio, # Use the extracted oversampling_ratio
                sampling_strategy='hybrid' # Using hybrid sampling (SMOTETomek)
            )
            print(f"    Training data after sampling: {len(resampled_texts)} samples", flush=True)

            train_dataset = ToxicDataset(
                resampled_texts, # Use resampled texts
                resampled_labels_df, # Use resampled labels
                self.tokenizer,
                Config.MAX_LENGTH,
                augment=True
            )

            val_dataset = ToxicDataset(
                fold_val_df['comment_text'],
                fold_val_df[Config.LABEL_COLS],
                self.tokenizer,
                Config.MAX_LENGTH
            )

            safety_model = self._create_model(trial)

            output_dir_fold = os.path.join(Config.GOOGLE_DRIVE_SAVE_BASE_PATH, f'results_trial_{trial.number}_fold_{fold}')
            os.makedirs(output_dir_fold, exist_ok=True)

            training_args = TrainingArguments(
                output_dir=output_dir_fold,
                eval_strategy="epoch",
                save_strategy="epoch",
                logging_strategy="epoch",
                per_device_eval_batch_size=params['per_device_train_batch_size'],
                save_total_limit=2,
                load_best_model_at_end=True,
                metric_for_best_model="eval_f1",
                greater_is_better=True,
                report_to="none",
                seed=Config.RANDOM_SEED,
                fp16=torch.cuda.is_available(),
                dataloader_pin_memory=False,
                learning_rate=params['learning_rate'],
                per_device_train_batch_size=params['per_device_train_batch_size'],
                num_train_epochs=params['num_train_epochs'],
                warmup_ratio=params['warmup_ratio'],
                weight_decay=params['weight_decay'],
                gradient_accumulation_steps=params['gradient_accumulation_steps'],
                lr_scheduler_type=params['lr_scheduler_type']
            )

            trainer = AdvancedTrainer(
                model=safety_model,
                args=training_args,
                train_dataset=train_dataset,
                eval_dataset=val_dataset,
                compute_metrics=self.metrics_calculator.compute_metrics,
                pos_weights=self.pos_weights,
                callbacks=[EarlyStoppingCallback(early_stopping_patience=Config.PATIENCE)],
            )

            try:
                trainer.train()
                eval_results = trainer.evaluate()
                # 'eval_f1' is correctly used as the primary metric, which your
                # compute_metrics function should set to be the macro F1 for optimization.
                fold_score = eval_results['eval_f1']

                print(f"    Fold {fold + 1} Results (Optimized Metric: Macro F1): {fold_score:.4f}", flush=True)
                print(f"      Macro F1 (from eval_results): {eval_results.get('eval_macro_f1', fold_score):.4f}", flush=True)
                print(f"      Micro F1: {eval_results.get('eval_micro_f1', 'N/A'):.4f}", flush=True)
                print(f"      Micro Precision: {eval_results.get('eval_micro_precision', 'N/A'):.4f}", flush=True)
                print(f"      Micro Recall: {eval_results.get('eval_micro_recall', 'N/A'):.4f}", flush=True)
                print(f"      Micro ROC-AUC: {eval_results.get('eval_micro_roc_auc', 'N/A'):.4f}", flush=True)

                self._plot_training_history(trainer.state.log_history, output_dir_fold, f"Fold {fold + 1}")

                cv_scores.append(fold_score)
            except Exception as e:
                # Improved error reporting for easier debugging
                print(f"    Fold {fold + 1} failed: {type(e).__name__}: {str(e)}", flush=True)
                cv_scores.append(0.0) # Append 0.0 or handle as per your Optuna study design

        mean_cv_score = np.mean(cv_scores)
        std_cv_score = np.std(cv_scores)

        print(f"\nTrial {trial.number} Results:", flush=True)
        print(f"    Mean CV Micro F1: {mean_cv_score:.4f} ± {std_cv_score:.4f}", flush=True)

        return mean_cv_score

    def run_optimization(self): # Renamed from optimize to run_optimization for clarity
        """
        Runs the Optuna hyperparameter optimization study for multi-label classification.
        Incorporates Hyperband pruning for efficient search.
        """
        print("Starting hyperparameter optimization...", flush=True)
        print(f"Search space: {Config.SEARCH_SPACE}", flush=True)

        # Determine max_resource for Hyperband Pruner based on your Config.N_TRAIN_EPOCHS range
        # Assuming Config.SEARCH_SPACE['num_train_epochs'] is the upper bound of your epochs search space
        max_epochs_for_pruner = Config.SEARCH_SPACE['num_train_epochs'][1]

        # Initialize Hyperband Pruner
        pruner = HyperbandPruner(
            min_resource=1,         # Pruning can start after the first epoch/step
            max_resource=max_epochs_for_pruner, # Max epochs a trial can run, matching your search space
            reduction_factor=3      # Common reduction factor for Hyperband
        )

        # Ensure the directory for the Optuna DB exists
        db_dir = os.path.dirname(Config.OPTUNA_DB_PATH.replace("sqlite:///", ""))
        os.makedirs(db_dir, exist_ok=True)
        print(f"Optuna DB directory ensured: {db_dir}", flush=True)

        study = optuna.create_study(
            direction='maximize',
            study_name='electra_toxic_multilabel_classification',
            storage=f'sqlite:///{Config.OPTUNA_DB_PATH}',
            load_if_exists=True,
            sampler=TPESampler(seed=Config.RANDOM_SEED), # Use TPESampler for efficient search
            pruner=pruner # Use the Hyperband Pruner
        )

        print(f"Running {Config.N_TRIALS} trials with Hyperband Pruner...", flush=True)
        study.optimize(self.objective, n_trials=Config.N_TRIALS, show_progress_bar=True)

        print("\nOptimization completed! ✨", flush=True)
        print(f"Best trial number: {study.best_trial.number}", flush=True)

        # Ensure this matches your metric_for_best_model, which should be 'macro_f1'
        print(f"Best Macro F1: {study.best_value:.4f}", flush=True)

        print("Best parameters found: 🚀", flush=True)
        for key, value in study.best_params.items():
            print(f"   {key}: {value}", flush=True)

        return study

    def get_optuna_summary_data(self, study):
        """
        Collects summary data for all Optuna trials.
        """
        summary_data = {
            "total_trials_completed": len(study.trials),
            "best_trial_number": study.best_trial.number,
            "best_objective_value": study.best_value,
            "best_hyperparameters": study.best_trial.params,
            "trial_history": []
        }

        # Collect summary for each trial
        for trial in study.trials:
            trial_info = {
                "trial_number": trial.number,
                "state": str(trial.state),
                "value": trial.value,
                "params": trial.params,
                "duration_seconds": trial.duration.total_seconds() if trial.duration else None
            }
            summary_data["trial_history"].append(trial_info)

        return summary_data

    def train_final_model(self, best_params, save_path=None):
        """
        Trains the final model using the best hyperparameters found during optimization.
        """
        if save_path is None:
            save_path = os.path.join(Config.GOOGLE_DRIVE_SAVE_BASE_PATH, "best_electra_multilabel_model")

        print("\nTraining final model with best parameters...", flush=True)

        # Apply sampling to the entire training set for the final model
        resampled_texts, resampled_labels_df = self._apply_sampling(
            X_train=self.train_df['comment_text'],
            y_train=self.train_df[Config.LABEL_COLS],
            oversampling_ratio=best_params.get('oversampling_ratio', 0.5), # Use optimized ratio or default
            sampling_strategy='hybrid'
        )
        print(f"Final training data after sampling: {len(resampled_texts)} samples", flush=True)


        train_dataset = ToxicDataset(
            resampled_texts,
            resampled_labels_df,
            self.tokenizer,
            Config.MAX_LENGTH,
            augment=True
        )

        val_dataset = None
        if self.val_df is not None:
            val_dataset = ToxicDataset(
                self.val_df['comment_text'],
                self.val_df[Config.LABEL_COLS], # Pass all labels for validation
                self.tokenizer,
                Config.MAX_LENGTH
            )

        safety_model = self._create_model()
        # Apply dropout parameters from best_params if they exist
        if 'hidden_dropout_prob' in best_params:
            safety_model.config.hidden_dropout_prob = best_params['hidden_dropout_prob']
        if 'attention_probs_dropout_prob' in best_params:
            safety_model.config.attention_probs_dropout_prob = best_params['attention_probs_dropout_prob']

        output_dir_final = os.path.join(Config.GOOGLE_DRIVE_SAVE_BASE_PATH, 'final_electra_multilabel_model_results')
        os.makedirs(output_dir_final, exist_ok=True)

        # Prepare TrainingArguments for the final training run
        training_args = TrainingArguments(
            output_dir=output_dir_final,
            eval_strategy="epoch" if val_dataset else "no",
            save_strategy="epoch",
            logging_strategy="epoch",
            per_device_train_batch_size=best_params.get('batch_size', 16),
            per_device_eval_batch_size=best_params.get('batch_size', 16),
            num_train_epochs=best_params.get('num_train_epochs', 3),
            learning_rate=best_params.get('learning_rate', 2e-5),
            warmup_ratio=best_params.get('warmup_ratio', 0.1),
            weight_decay=best_params.get('weight_decay', 0.01),
            gradient_accumulation_steps=best_params.get('gradient_accumulation_steps', 1),
            lr_scheduler_type=best_params.get('lr_scheduler_type', 'linear'),
            save_total_limit=3, # Keep up to 3 best checkpoints
            load_best_model_at_end=True if val_dataset else False,
            metric_for_best_model="eval_f1" if val_dataset else None, # Micro F1 for multilabel
            greater_is_better=True,
            report_to="none",
            seed=Config.RANDOM_SEED,
            fp16=torch.cuda.is_available(),
            dataloader_pin_memory=False,
        )

        trainer = AdvancedTrainer(
            model=safety_model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=self.metrics_calculator.compute_metrics,
            pos_weights=self.pos_weights, # Pass tensor of pos_weights
            callbacks=[EarlyStoppingCallback(early_stopping_patience=Config.PATIENCE)] if val_dataset else [],
        )

        trainer.train()

        if val_dataset:
            final_eval_results = trainer.evaluate()
            print("\nFinal Model Evaluation Results:", flush=True)
            for k, v in final_eval_results.items():
                if k.startswith('eval_'): # Print metrics that were evaluated
                    print(f"    {k.replace('eval_', '')}: {v:.4f}", flush=True)
                elif k.endswith('_f1') or k.endswith('_precision') or k.endswith('_recall') or k.endswith('_roc_auc') or k.endswith('_pr_auc'):
                    print(f"    {k}: {v:.4f}", flush=True)


            # Get true labels and predicted probabilities from the final evaluation
            predictions = trainer.predict(val_dataset)
            y_true_final = predictions.label_ids
            y_pred_proba_final = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()

            # Optimize thresholds on the final validation set for comprehensive evaluation
            print("\nOptimizing classification thresholds for final model evaluation...", flush=True)
            best_thresholds_final, best_threshold_score_final = self.metrics_calculator.optimize_thresholds(
                y_true_final, y_pred_proba_final, n_trials=100
            )
            print(f"Optimized macro F1 (threshold tuning): {best_threshold_score_final:.4f}", flush=True)
            print("Optimized thresholds:", flush=True)
            for i, label_name in enumerate(Config.LABEL_COLS):
                print(f"    {best_thresholds_final[i]:.3f}", flush=True) # Corrected variable name

            # Re-evaluate with optimized thresholds to get final confusion matrices and plots
            final_binary_preds = (y_pred_proba_final > np.array(best_thresholds_final)).astype(int)

            self._plot_training_history(trainer.state.log_history, output_dir_final, "Final Model")
            self._plot_multilabel_confusion_matrix(y_true_final, final_binary_preds, Config.LABEL_COLS, output_dir_final, "Final Model")
            self._plot_precision_recall_curve_multilabel(y_true_final, y_pred_proba_final, Config.LABEL_COLS, output_dir_final, "Final Model")
            self._plot_roc_curve_multilabel(y_true_final, y_pred_proba_final, Config.LABEL_COLS, output_dir_final, "Final Model")
            self._plot_probability_distribution_multilabel(y_true_final, y_pred_proba_final, Config.LABEL_COLS, output_dir_final, "Final Model")

            # Save the optimized thresholds to a CSV file
            threshold_df = pd.DataFrame({
                'label': Config.LABEL_COLS,
                'threshold': best_thresholds_final
            })
            threshold_df.to_csv(os.path.join(Config.GOOGLE_DRIVE_SAVE_BASE_PATH, 'optimized_thresholds_multilabel.csv'), index=False)
            print(f"Final optimized thresholds saved to {os.path.join(Config.GOOGLE_DRIVE_SAVE_BASE_PATH, 'optimized_thresholds_multilabel.csv')}", flush=True)


        trainer.save_model(save_path)
        self.tokenizer.save_pretrained(save_path)

        print(f"Final model saved to {save_path}", flush=True)

        # Save the training log history for the final model
        log_history_path = os.path.join(output_dir_final, 'final_model_training_log_history.json')
        with open(log_history_path, 'w') as f:
            json.dump(trainer.state.log_history, f, indent=4)
        print(f"Final model training log history saved to {log_history_path}", flush=True)


        return trainer


# Helper function for LIME predictions (needs to be outside classes to be easily callable by LIME)
def predict_proba_for_lime(texts, model, tokenizer, max_length, device):
    """
    Predictor function required by LIME. Takes a list of raw texts and returns
    prediction probabilities for each class.
    """
    inputs = tokenizer(
        texts,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'
    ).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.sigmoid(outputs.logits).cpu().numpy()
    return probabilities

def main():
    set_seed(Config.RANDOM_SEED) # Ensure reproducibility

    # Create the base directory for saving results
    os.makedirs(Config.GOOGLE_DRIVE_SAVE_BASE_PATH, exist_ok=True)
    print(f"Google Drive base save path ensured: {Config.GOOGLE_DRIVE_SAVE_BASE_PATH}", flush=True)

    print("Loading data...", flush=True)
    try:
        # Load your datasets as specified
        train_df = pd.read_csv("train.csv")
        test_df = pd.read_csv("test.csv")
        test_labels = pd.read_csv("test_labels.csv")
        sample_submission = pd.read_csv("sample_submission.csv")

        print("Data loaded successfully.", flush=True)
        print(f"Train DataFrame shape: {train_df.shape}", flush=True)
        print(f"Test DataFrame shape: {test_df.shape}", flush=True)
        print(f"Test Labels DataFrame shape: {test_labels.shape}", flush=True)
        print(f"Sample Submission DataFrame shape: {sample_submission.shape}", flush=True)


    except FileNotFoundError:
        print("Error: One or more dataset files (train.csv, test.csv, test.csv, sample_submission.csv) not found.", flush=True)
        print("Please ensure all required CSV files are in the same directory as the script.", flush=True)
        return

    # Fill NaN values in 'comment_text'
    train_df['comment_text'] = train_df['comment_text'].fillna('')
    test_df['comment_text'] = test_df['comment_text'].fillna('') # Also fill for test_df if needed

    # --- Use all original labels directly for multilabel classification ---
    # Ensure all label columns are present
    if not all(col in train_df.columns for col in Config.LABEL_COLS):
        print(f"Error: One or more label columns {Config.LABEL_COLS} not found in the training dataset. Exiting.", flush=True)
        return

    print(f"\nLabel columns being used for training: {Config.LABEL_COLS}", flush=True)

    # Display class distribution for each label (important for multilabel)
    print("\nMultilabel Class distribution (Positive count and percentage in training data):", flush=True)
    for col in Config.LABEL_COLS:
        pos_count = train_df[col].sum()
        total_count = len(train_df)
        pos_pct = (pos_count / total_count) * 100
        print(f"  '{col}': {pos_count:,} positive samples ({pos_pct:.2f}%)", flush=True)

    # Splitting the main training data into a training and validation set
    # This validation set is used for the *final model training* and evaluation,
    # distinct from the CV folds within Optuna.
    print("\nSplitting training data into train and validation sets (80/20 split)...", flush=True)
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        train_df['comment_text'],
        train_df[Config.LABEL_COLS],
        test_size=0.2,
        random_state=Config.RANDOM_SEED,
        stratify=train_df['toxic'] # Stratify by 'toxic' label for a more balanced split
    )

    # Reconstruct dataframes for ModelOptimizer
    train_df_for_optimizer = pd.DataFrame({'comment_text': train_texts}).reset_index(drop=True)
    train_labels_for_optimizer = train_labels.reset_index(drop=True)
    val_df_for_optimizer = pd.DataFrame({'comment_text': val_texts}).reset_index(drop=True)
    val_labels_for_optimizer = val_labels.reset_index(drop=True)

    # Merge texts and labels back for convenience in ModelOptimizer initialization
    # (ModelOptimizer expects the full train_df with 'comment_text' and all label columns)
    train_combined_df = pd.concat([train_df_for_optimizer, train_labels_for_optimizer], axis=1)
    val_combined_df = pd.concat([val_df_for_optimizer, val_labels_for_optimizer], axis=1) # Pass this to ModelOptimizer

    print(f"Training set size for final model: {len(train_combined_df)}", flush=True)
    print(f"Validation set size for final model: {len(val_combined_df)}", flush=True)

    optimizer = ModelOptimizer(train_combined_df, val_combined_df) # Pass val_combined_df here

    best_params = {}
    study = None # Initialize study to None
    if Config.LOAD_EXISTING_STUDY_RESULTS:
        print(f"\nAttempting to load existing Optuna study from {Config.OPTUNA_DB_PATH}...", flush=True)
        try:
            study = optuna.load_study(
                study_name='electra_toxic_multilabel_classification',
                storage=f'sqlite:///{Config.OPTUNA_DB_PATH}'
            )
            best_params = study.best_trial.params
            print("Successfully loaded best parameters from existing study:", flush=True)
            for k, v in best_params.items():
                print(f"    {k}: {v}", flush=True)
        except Exception as e:
            print(f"Could not load existing study: {e}. Starting new optimization.", flush=True)
            Config.LOAD_EXISTING_STUDY_RESULTS = False # Fallback to running optimization

    if not Config.LOAD_EXISTING_STUDY_RESULTS:
        study = optimizer.run_optimization() # Changed from .optimize() to .run_optimization()
        best_params = study.best_trial.params

    # Train the final model with the best parameters
    final_trainer = optimizer.train_final_model(best_params)

    print("\nTraining and optimization complete! ", flush=True)
    print(f"Results, model checkpoints, and plots are saved in: {Config.GOOGLE_DRIVE_SAVE_BASE_PATH}", flush=True)

    # --- Generate Optuna Summary Data ---
    if study: # Only generate summary if a study object is available
        optuna_summary = optimizer.get_optuna_summary_data(study)
        print("\n--- Optuna Optimization Summary Data ---", flush=True)
        print(json.dumps(optuna_summary, indent=2), flush=True)
        print("\n--- End of Optuna Optimization Summary Data ---", flush=True)
    else:
        print("\nSkipping Optuna summary data generation as no study object was available.", flush=True)


    # --- Explainability (XAI) Section using LIME ---
    print("\n--- Starting Explainable AI (XAI) Analysis with LIME ---", flush=True)

    # Load the best trained model and tokenizer for XAI
    final_model_path = os.path.join(Config.GOOGLE_DRIVE_SAVE_BASE_PATH, "best_electra_multilabel_model")
    try:
        final_model = AutoModelForSequenceClassification.from_pretrained(final_model_path).to(Config.DEVICE)
        final_tokenizer = AutoTokenizer.from_pretrained(final_model_path)
        print(f"Loaded final model and tokenizer from {final_model_path}", flush=True)
    except Exception as e:
        print(f"Error loading final model for XAI: {e}. Skipping XAI analysis.", flush=True)
        final_model = None # Set to None to prevent further errors if loading failed

    if final_model:
        # Prepare the predictor function for LIME
        lime_predictor = lambda texts: predict_proba_for_lime(
            texts, final_model, final_tokenizer, Config.MAX_LENGTH, Config.DEVICE
        )

        # Initialize LIME Explainer
        explainer = LimeTextExplainer(class_names=Config.LABEL_COLS)

        # Select a few examples for explanation from the validation set
        # You can adjust the number of examples or criteria for selection
        num_xai_examples = min(5, len(val_combined_df)) # Explain up to 5 examples from validation set
        xai_examples_df = val_combined_df.sample(n=num_xai_examples, random_state=Config.RANDOM_SEED).reset_index(drop=True)

        for idx in range(num_xai_examples):
            example_comment_text = xai_examples_df['comment_text'].iloc[idx]
            true_labels = xai_examples_df[Config.LABEL_COLS].iloc[idx].values

            # Get model's raw probabilities for this example
            model_probabilities = lime_predictor([example_comment_text])[0]

            print(f"\n--- XAI Example {idx + 1} ---", flush=True)
            print(f"Comment: '{example_comment_text}'", flush=True)
            print(f"True Labels: {true_labels}", flush=True)
            print(f"Model Probabilities: {model_probabilities.round(3)}", flush=True)

            # Generate explanation for each relevant label
            for i, label_name in enumerate(Config.LABEL_COLS):
                # Decide which labels to explain: e.g., true positive, or high predicted probability
                if true_labels[i] == 1 or model_probabilities[i] > 0.5:
                    print(f"\n  Explanation for label: '{label_name}' (True: {true_labels[i]}, Predicted Prob: {model_probabilities[i]:.3f})", flush=True)
                    try:
                        # Attempt to generate explanation
                        exp = explainer.explain_instance(
                            example_comment_text,
                            lime_predictor,
                            labels=[i],  # Request explanation specifically for this label's index
                            num_features=10,
                            num_samples=1000
                        )

                        # Check if the explanation object is valid and has content for the label
                        if exp and hasattr(exp, 'as_list'):
                            explanation_list = exp.as_list(label=i)
                            if explanation_list:
                                print(explanation_list, flush=True)
                            else:
                                print(f"  No LIME explanation features found for label '{label_name}' (label index {i}). This can happen if the model's prediction for this label is very stable across perturbed samples.", flush=True)
                        else:
                            print(f"  LIME explainer returned an invalid explanation object for label '{label_name}' (label index {i}).", flush=True)

                    except Exception as e:
                        # Catch any other unexpected errors during LIME explanation
                        print(f"  An unexpected error occurred during LIME explanation for label '{label_name}' (label index {i}): {type(e).__name__}: {str(e)}", flush=True)
                else:
                    print(f"  Skipping explanation for label '{label_name}' (True: {true_labels[i]}, Predicted Prob: {model_probabilities[i]:.3f} - not a strong prediction or true positive)", flush=True)

    print("\n--- XAI Analysis Complete ---", flush=True)

if __name__ == '__main__':
    # Mount Google Drive if running in Colab
    # from google.colab import drive
    # drive.mount('/content/drive')
    # print("\nGoogle Drive mounted!", flush=True)

    main()


Google Drive base save path ensured: /content/drive/MyDrive/my_electra_multilabel_classifier_results
Loading data...
Data loaded successfully.
Train DataFrame shape: (159571, 8)
Test DataFrame shape: (153164, 2)
Test Labels DataFrame shape: (153164, 7)
Sample Submission DataFrame shape: (153164, 7)

Label columns being used for training: ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

Multilabel Class distribution (Positive count and percentage in training data):
  'toxic': 15,294 positive samples (9.58%)
  'severe_toxic': 1,595 positive samples (1.00%)
  'obscene': 8,449 positive samples (5.29%)
  'threat': 478 positive samples (0.30%)
  'insult': 7,877 positive samples (4.94%)
  'identity_hate': 1,405 positive samples (0.88%)

Splitting training data into train and validation sets (80/20 split)...
Training set size for final model: 127656
Validation set size for final model: 31915
Positive weight for 'toxic': 9.43
Positive weight for 'severe_toxic': 98.42
P

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1,Toxic F1,Toxic Precision,Toxic Recall,Toxic Roc Auc,Toxic Pr Auc,Toxic Tn,Toxic Fp,Toxic Fn,Toxic Tp,Severe Toxic F1,Severe Toxic Precision,Severe Toxic Recall,Severe Toxic Roc Auc,Severe Toxic Pr Auc,Severe Toxic Tn,Severe Toxic Fp,Severe Toxic Fn,Severe Toxic Tp,Obscene F1,Obscene Precision,Obscene Recall,Obscene Roc Auc,Obscene Pr Auc,Obscene Tn,Obscene Fp,Obscene Fn,Obscene Tp,Threat F1,Threat Precision,Threat Recall,Threat Roc Auc,Threat Pr Auc,Threat Tn,Threat Fp,Threat Fn,Threat Tp,Insult F1,Insult Precision,Insult Recall,Insult Roc Auc,Insult Pr Auc,Insult Tn,Insult Fp,Insult Fn,Insult Tp,Identity Hate F1,Identity Hate Precision,Identity Hate Recall,Identity Hate Roc Auc,Identity Hate Pr Auc,Identity Hate Tn,Identity Hate Fp,Identity Hate Fn,Identity Hate Tp,Micro F1,Micro Precision,Micro Recall,Micro Roc Auc,Macro F1,Macro Precision,Macro Recall,Macro Roc Auc
1,0.608100,0.590379,0.645714,0.808664,0.724263,0.915332,0.987858,0.914454,27790,1066,259,2800,0.452830,0.320427,0.771704,0.988627,0.422795,31095,509,71,240,0.773661,0.648595,0.958480,0.993527,0.900544,29317,888,71,1639,0.578947,0.503817,0.680412,0.977405,0.464883,31753,65,31,66,0.731134,0.606551,0.920126,0.988642,0.811740,29376,949,127,1463,0.529048,0.411538,0.740484,0.987191,0.574177,31320,306,75,214,0.976934,0.976934,0.976934,0.990926,0.645714,0.535865,0.831090,0.987209
2,0.302900,0.627065,0.647134,0.793801,0.692739,0.929389,0.986835,0.909787,27595,1261,216,2843,0.477273,0.351598,0.742765,0.989455,0.449779,31178,426,80,231,0.811999,0.727863,0.918129,0.993558,0.895570,29618,587,140,1570,0.545455,0.442308,0.711340,0.979765,0.483205,31731,87,28,69,0.741230,0.622014,0.916981,0.988653,0.799376,29439,886,132,1458,0.513043,0.374010,0.816609,0.988585,0.572882,31231,395,53,236,0.977592,0.977592,0.977592,0.991236,0.647134,0.535088,0.839202,0.987809
3,0.205500,0.887501,0.663888,0.792669,0.692834,0.926120,0.984219,0.886975,27600,1256,226,2833,0.493082,0.404959,0.630225,0.989108,0.441843,31316,288,115,196,0.821238,0.755152,0.900000,0.992221,0.891017,29706,499,171,1539,0.546154,0.435583,0.731959,0.980264,0.470178,31726,92,26,71,0.757559,0.663671,0.882390,0.987728,0.792052,29614,711,187,1403,0.572626,0.480094,0.709343,0.985552,0.562001,31404,222,84,205,0.979754,0.979754,0.979754,0.990480,0.663888,0.572049,0.796673,0.986515
4,0.145100,1.006753,0.667716,0.816482,0.738177,0.913370,0.985344,0.900385,27865,991,265,2794,0.483173,0.385797,0.646302,0.988721,0.421466,31284,320,110,201,0.822872,0.754634,0.904678,0.991831,0.886228,29702,503,163,1547,0.541833,0.441558,0.701031,0.978552,0.455903,31732,86,29,68,0.759804,0.670029,0.877358,0.987401,0.776451,29638,687,195,1395,0.582133,0.498765,0.698962,0.982079,0.561049,31423,203,87,202,0.980996,0.980996,0.980996,0.990346,0.667716,0.581493,0.790284,0.985655



Final Model Evaluation Results:
    f1: 0.6677
    loss: 1.0068
    toxic_f1: 0.8165
    toxic_precision: 0.7382
    toxic_recall: 0.9134
    toxic_roc_auc: 0.9853
    toxic_pr_auc: 0.9004
    toxic_tn: 27865.0000
    toxic_fp: 991.0000
    toxic_fn: 265.0000
    toxic_tp: 2794.0000
    severe_toxic_f1: 0.4832
    severe_toxic_precision: 0.3858
    severe_toxic_recall: 0.6463
    severe_toxic_roc_auc: 0.9887
    severe_toxic_pr_auc: 0.4215
    severe_toxic_tn: 31284.0000
    severe_toxic_fp: 320.0000
    severe_toxic_fn: 110.0000
    severe_toxic_tp: 201.0000
    obscene_f1: 0.8229
    obscene_precision: 0.7546
    obscene_recall: 0.9047
    obscene_roc_auc: 0.9918
    obscene_pr_auc: 0.8862
    obscene_tn: 29702.0000
    obscene_fp: 503.0000
    obscene_fn: 163.0000
    obscene_tp: 1547.0000
    threat_f1: 0.5418
    threat_precision: 0.4416
    threat_recall: 0.7010
    threat_roc_auc: 0.9786
    threat_pr_auc: 0.4559
    threat_tn: 31732.0000
    threat_fp: 86.0000
    threat_fn: 2

[I 2025-07-09 22:45:28,513] A new study created in memory with name: multilabel_threshold_optimization


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-07-09 22:45:28,536] Trial 0 finished with value: 0.6709520813971118 and parameters: {'threshold_toxic': 0.4547605488666665, 'threshold_severe_toxic': 0.524701025941887, 'threshold_obscene': 0.5879247327353317, 'threshold_threat': 0.5103572445858705, 'threshold_insult': 0.874816273954321, 'threshold_identity_hate': 0.6095196155204955}. Best is trial 0 with value: 0.6709520813971118.
[I 2025-07-09 22:45:28,549] Trial 1 finished with value: 0.6622658221236178 and parameters: {'threshold_toxic': 0.8801252346468774, 'threshold_severe_toxic': 0.16463014965786815, 'threshold_obscene': 0.10256780644602169, 'threshold_threat': 0.13121456555077968, 'threshold_insult': 0.3911755571747306, 'threshold_identity_hate': 0.3679013112678855}. Best is trial 0 with value: 0.6709520813971118.
[I 2025-07-09 22:45:28,563] Trial 2 finished with value: 0.6691552204620769 and parameters: {'threshold_toxic': 0.7250511737010551, 'threshold_severe_toxic': 0.6494576250283648, 'threshold_obscene': 0.11493707